In [1]:
# import basic modules
import sys
import os
import time
import numpy as np
from tqdm import tqdm
import gc
import torch
import argparse
import skimage.transform

# import custom modules
root_dir   = '/user_data/mmhender/imStat/'
sys.path.append(os.path.join(root_dir, 'code'))
from model_src import fwrf_fit as fwrf_fit
from model_src import fwrf_predict as fwrf_predict
from model_src import texture_statistics_gabor, texture_statistics_pyramid, bdcn_features
from utils import nsd_utils, roi_utils
from model_fitting import initialize_fitting


bdcn_path = '/user_data/mmhender/toolboxes/BDCN/'


fpX = np.float32

In [3]:
def fun(a):
    
    return a+2



In [4]:
thing = fun
thing(3)

5

In [2]:
subject=1
volume_space = True

ridge=True

shuffle_images=False
random_images=False
random_voxel_data=False

sample_batch_size=100
voxel_batch_size=100
zscore_features=True

up_to_sess=1
debug=True
shuff_rnd_seed=0

n_ori=4
n_sf=4
nonlin_fn=False

# fitting_type='pyramid_texture'

do_fitting=True
do_val=True
do_varpart=True

date_str=None

In [3]:
device = initialize_fitting.init_cuda()
model_name, feature_types_exclude = initialize_fitting.get_pyramid_model_name(ridge, n_ori, n_sf)
    

#device: 1
device#: 0
device name: GeForce RTX 2080 Ti

torch: 1.8.1+cu111
cuda:  11.1
cudnn: 8005
dtype: torch.float32


In [4]:
# decide what voxels to use  
voxel_mask, voxel_index, voxel_roi, voxel_ncsnr, brain_nii_shape = roi_utils.get_voxel_roi_info(subject, volume_space)

sessions = np.arange(0,up_to_sess)
zscore_betas_within_sess = True
# get all data and corresponding images, in two splits. always fixed set that gets left out
trn_stim_data, trn_voxel_data, val_stim_data, val_voxel_data, image_order = nsd_utils.get_data_splits(subject, sessions=sessions, \
                                                                 voxel_mask=voxel_mask, volume_space=volume_space, \
                                                                  zscore_betas_within_sess=zscore_betas_within_sess, \
                                                                  shuffle_images=shuffle_images, random_images=random_images, \
                                                                                     random_voxel_data=random_voxel_data)


Volume space: ROI defs are located at: /lab_data/tarrlab/common/datasets/NSD/nsddata/ppdata/subj01/func1pt8mm/roi

3794 voxels of overlap between kastner and prf definitions, using prf defs
unique values in retino labels:
[-1.  0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16.
 17. 18. 19. 20. 21. 22. 23. 24. 25.]
0 voxels of overlap between face and place definitions, using place defs
unique values in categ labels:
[-1.  0. 26. 27. 28. 30. 31. 32. 33.]
1535 voxels are defined (differently) in both retinotopic areas and category areas

14913 voxels are defined across all areas, and will be used for analysis

Loading numerical label/name mappings for all ROIs:
[1, 2, 3, 4, 5, 6, 7]
['V1v', 'V1d', 'V2v', 'V2d', 'V3v', 'V3d', 'hV4']
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]
['V1v', 'V1d', 'V2v', 'V2d', 'V3v', 'V3d', 'hV4', 'VO1', 'VO2', 'PHC1', 'PHC2', 'TO2', 'TO1', 'LO2', 'LO1', 'V3B', 'V3A', 'IPS0', 'IPS1', 'IPS2', 'IPS

In [5]:
# Need a multiple of 8
process_at_size=240
trn_stim_data = skimage.transform.resize(trn_stim_data, output_shape=(trn_stim_data.shape[0],1,process_at_size, process_at_size))
val_stim_data = skimage.transform.resize(val_stim_data, output_shape=(val_stim_data.shape[0],1,process_at_size, process_at_size))
    

In [6]:
# Set up the pRFs to test
aperture_rf_range = 1.1
aperture, models = initialize_fitting.get_prf_models(aperture_rf_range=aperture_rf_range)    

# More params for fitting
holdout_size, lambdas = initialize_fitting.get_fitting_pars(trn_voxel_data, zscore_features, ridge=ridge)


most extreme RF positions:
[-0.55 -0.55  0.04]
[0.55       0.55       0.40000001]

Possible lambda values are:
[1.0000000e+00 4.2169652e+00 1.7782795e+01 7.4989418e+01 3.1622775e+02
 1.3335215e+03 5.6234131e+03 2.3713736e+04 1.0000000e+05]


In [698]:
# Set up the pyramid
_fmaps_fn = texture_statistics_pyramid.steerable_pyramid_extractor(pyr_height=n_sf, n_ori = n_ori)
# Params for the spatial aspect of the model (possible pRFs)
#     aperture_rf_range=0.8 # using smaller range here because not sure what to do with RFs at edges...
aperture_rf_range = 1.1
aperture, models = initialize_fitting.get_prf_models(aperture_rf_range=aperture_rf_range)    

do_varpart=True
group_all_hl_feats=True
# Initialize the "texture" model which builds on first level feature maps
n_prf_sd_out=2
_texture_fn = texture_feature_extractor(_fmaps_fn,sample_batch_size=sample_batch_size, feature_types_exclude=feature_types_exclude, \
                                        n_prf_sd_out=n_prf_sd_out, aperture=aperture, 
                                        do_varpart = do_varpart, group_all_hl_feats = group_all_hl_feats, device=device)
# _texture_fn_old = texture_statistics_pyramid.texture_feature_extractor(_fmaps_fn,sample_batch_size=sample_batch_size, \
#                                                                        feature_types_exclude=feature_types_exclude, \
#                                                                        n_prf_sd_out=n_prf_sd_out, aperture=aperture, device=device)

  

most extreme RF positions:
[-0.55 -0.55  0.04]
[0.55       0.55       0.40000001]
[]


In [699]:
# Set up the contour model
map_ind = -1
n_prf_sd_out=2
mult_patch_by_prf=True
downsample_factor=1
do_nms=False
do_pca=True
max_pc_to_retain = 400
min_pct_var=99
# Set up the contour feature extractor
pretrained_model_file = os.path.join(bdcn_path,'params','bdcn_pretrained_on_bsds500.pth')
_contour_fn = bdcn_feature_extractor(pretrained_model_file, device, aperture, n_prf_sd_out, \
                                           batch_size=10, map_ind=map_ind, mult_patch_by_prf=mult_patch_by_prf,                                     
                                            downsample_factor = downsample_factor, do_nms = do_nms, \
                                            do_pca = do_pca, min_pct_var = min_pct_var, max_pc_to_retain = max_pc_to_retain)


/user_data/mmhender/toolboxes/BDCN/bdcn.py:232: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  nn.init.constant(param, 0.080)


In [704]:
# _feature_extractor = combined_feature_extractor(modules=[_texture_fn, _contour_fn], module_names=['bdcn1','texture'], \
#                                                do_varpart = True)
_feature_extractor = combined_feature_extractor(modules=[_contour_fn, _texture_fn], module_names=['bdcn', 'texture'], \
                                               do_varpart = True)

In [705]:
images = trn_stim_data
image_size = images.shape[2:4]
dtype = images.dtype.type
_feature_extractor.init_for_fitting(image_size, models, dtype)

Initializing for fitting: finding the size of feature matrix for each candidate prf
Initializing arrays for PCA params
Clearing BDCN contour features from memory.
Clearing steerable pyramid features from memory.


/home/mmhender/myenv/lib/python3.7/site-packages/ipykernel_launcher.py:80: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


In [573]:
masks, names = _feature_extractor.get_partial_versions()

In [517]:
_texture_fn.feature_group_names

['mean_magnitudes', 'all_other_texture_feats']

In [574]:
features, feature_inds_defined = _feature_extractor(images, models[0,:], 0, fitting_mode=True)

Running BDCN contour feature extraction...
Images array shape is:
(688, 1, 240, 240)
Final array shape is:
torch.Size([688, 1, 240, 240])
time elapsed = 7.06344
Getting features for pRF [x,y,sigma]:
[-0.55, -0.55, 0.03999999910593033]
bbox to crop is:
[232, 240, 0, 9]
[min max] of first image patch is:
[tensor(8.4639e-06, device='cuda:0'), tensor(0.2264, device='cuda:0')]
Preparing for PCA: original dims of features:
(688, 72)
Running PCA...
Retaining 9 components to expl 99 pct var
Running steerable pyramid feature extraction...
Images array shape is:
(688, 1, 240, 240)
time elapsed = 135.12133
Computing higher order correlations...
time elapsed = 7.16783
Final size of features concatenated is [688 x 641]
Feature types included are:
['pixel_stats', 'mean_magnitudes', 'mean_realparts', 'marginal_stats_lowpass_recons', 'variance_highpass_resid', 'magnitude_feature_autocorrs', 'lowpass_recon_autocorrs', 'highpass_resid_autocorrs', 'magnitude_within_scale_crosscorrs', 'real_within_scale_c

In [566]:
features, feature_inds_defined = _contour_fn(images, models[0,:], 0, fitting_mode=True)

Running BDCN contour feature extraction...
Images array shape is:
(688, 1, 240, 240)
Final array shape is:
torch.Size([688, 1, 240, 240])
time elapsed = 7.07141
Getting features for pRF [x,y,sigma]:
[-0.55, -0.55, 0.03999999910593033]
bbox to crop is:
[232, 240, 0, 9]
[min max] of first image patch is:
[tensor(8.4639e-06, device='cuda:0'), tensor(0.2264, device='cuda:0')]
Preparing for PCA: original dims of features:
(688, 72)
Running PCA...
Retaining 9 components to expl 99 pct var


In [575]:
features.shape

torch.Size([688, 650])

In [583]:
inds = np.concatenate((feature_inds_defined, [True]), axis=0)
inds[1000:]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True])

In [710]:
# best_params

In [703]:
class combined_feature_extractor(torch.nn.Module):
    
    def __init__(self, modules, module_names, do_varpart=False):
        
        super(combined_feature_extractor, self).__init__()
        self.modules = modules
        self.module_names = module_names
        self.do_varpart = do_varpart
        
    def init_for_fitting(self, image_size, models, dtype):
        
        max_features = 0
        for module in self.modules:            
            module.init_for_fitting(image_size, models, dtype)
            max_features += module.max_features
            
        self.max_features = max_features
        
    def clear_maps(self):
        
        for module in self.modules:
            module.clear_maps()
            
    def get_partial_versions(self):
        
        if not hasattr(self, 'max_features'):
            raise RuntimeError('need to run init_for_fitting first')

        n_total_feat = _feature_extractor.max_features
        masks = np.ones((1,n_total_feat), dtype=int)
        names = ['full_combined_model']
            
        if self.do_varpart:

            # going to define "masks" that combine certain sub-sets of models features at a time
            # to be uses for variance partitioning
            feature_start_ind = 0

            for mi, module in enumerate(_feature_extractor.modules):

                # first a version that only includes the features in current module
                new_mask = np.zeros((1, n_total_feat), dtype=int)
                new_mask[0,feature_start_ind:feature_start_ind+module.max_features] = 1
                masks = np.concatenate((masks, new_mask), axis=0)
                names += ['just_' + _feature_extractor.module_names[mi]]

                if len(_feature_extractor.modules)>2:        
                    # next a version that only everything but the features in current module
                    # (note if there are just 2 modules, this would be redundant)
                    new_mask = np.ones((1, n_total_feat), dtype=int)
                    new_mask[0,feature_start_ind:feature_start_ind+module.max_features] = 0
                    masks = np.concatenate((masks, new_mask), axis=0)
                    names += ['leave_out_' + _feature_extractor.module_names[mi]]

                # if the module has any subsets of features defined, will also do partial versions with those subsets only
                module_partial_masks, module_partial_names = module.get_partial_versions()
                if len(module_partial_names)>1:        
                    new_masks = np.zeros((len(module_partial_names)-1, n_total_feat), dtype=int)
                    new_masks[:,feature_start_ind:feature_start_ind+module.max_features] = module_partial_masks[1:]
                    masks = np.concatenate((masks, new_masks), axis=0)
                    names += [_feature_extractor.module_names[mi] + '_' + name + '_no_other_modules'for name in module_partial_names[1:]]

                    if len(module_partial_names)==3:        
                        # for this special case, also adding in some other combinations  
                        # if more than two subsets then this will get too complicated...
                        new_masks = np.ones((len(module_partial_names)-1, n_total_feat), dtype=int)
                        new_masks[:,feature_start_ind:feature_start_ind+module.max_features] = module_partial_masks[1:]
                        masks = np.concatenate((masks, new_masks), axis=0)
                        names += [_feature_extractor.module_names[mi] + '_' + name + '_plus_other_modules' for name in module_partial_names[1:]]

                feature_start_ind += module.max_features

        return masks, names
        
    def forward(self, images, prf_params, prf_model_ind, fitting_mode = True):

        for mi, module in enumerate(self.modules):
            
            features, inds = module(images, prf_params, prf_model_ind, fitting_mode)
            
            if mi==0:
                all_features_concat = features
                feature_inds_defined = inds
            else:
                all_features_concat = torch.cat((all_features_concat, features), axis=1)
                feature_inds_defined = np.concatenate((feature_inds_defined, inds), axis=0)
  
        return all_features_concat, feature_inds_defined

In [1012]:
_feature_extractor.modules[1].feature_types_include

['pixel_stats',
 'mean_magnitudes',
 'mean_realparts',
 'marginal_stats_lowpass_recons',
 'variance_highpass_resid',
 'magnitude_feature_autocorrs',
 'lowpass_recon_autocorrs',
 'highpass_resid_autocorrs',
 'magnitude_within_scale_crosscorrs',
 'real_within_scale_crosscorrs',
 'magnitude_across_scale_crosscorrs',
 'real_imag_across_scale_crosscorrs',
 'real_spatshift_within_scale_crosscorrs',
 'real_spatshift_across_scale_crosscorrs']

In [557]:
import numpy as np
import torch
import time
from collections import OrderedDict
import torch.nn as nn
import pyrtools as pt
from utils import numpy_utils, torch_utils, texture_utils, prf_utils

class texture_feature_extractor(nn.Module):
    
    """
    Module to compute higher-order texture statistics of input images (e.g. Portilla & Simoncelli 2000, IJCV)
    Statistics are computed within a specified region of space (a voxel's pRF)
    Can specify different subsets of features to include (i.e. pixel-level stats, simple/complex cells, cross-correlations, auto-correlations)
    Inputs to the forward pass are images and pRF parameters of interest [x,y,sigma]
    """
    
    def __init__(self,_fmaps_fn, sample_batch_size=100, feature_types_exclude=None, n_prf_sd_out=2, \
                 aperture=1.0, do_varpart=False, group_all_hl_feats=False, device=None):
        
        super(texture_feature_extractor, self).__init__()
        
        self.fmaps_fn = _fmaps_fn   
        self.fmaps = None
        self.n_sf = _fmaps_fn.pyr_height
        self.n_ori =  _fmaps_fn.n_ori
       
        self.sample_batch_size = sample_batch_size       
        self.n_prf_sd_out = n_prf_sd_out
        self.aperture = aperture
        self.device = device       
        
        self.do_varpart = do_varpart
        self.group_all_hl_feats = group_all_hl_feats
        
        self.update_feature_list(feature_types_exclude)
        self.do_pca = False
       
    def init_for_fitting(self, image_size, models, dtype):

        """
        Additional initialization operations.
        """
       
        self.max_features = self.n_features_total            
        self.clear_maps()
        
       
    def update_feature_list(self, feature_types_exclude):
        
        feature_types_all = ['pixel_stats', 'mean_magnitudes', 'mean_realparts', 'marginal_stats_lowpass_recons', 'variance_highpass_resid', \
            'magnitude_feature_autocorrs', 'lowpass_recon_autocorrs', 'highpass_resid_autocorrs', \
            'magnitude_within_scale_crosscorrs', 'real_within_scale_crosscorrs', 'magnitude_across_scale_crosscorrs', 'real_imag_across_scale_crosscorrs', \
            'real_spatshift_within_scale_crosscorrs', 'real_spatshift_across_scale_crosscorrs']
        feature_type_dims = [6,16,16,10,1,\
                        272,73,25,\
                        24,24,48,96,\
                       10,20]

        if feature_types_exclude is None:
            feature_types_exclude = []
        # decide which features to ignore, or use all features
        self.feature_types_exclude = feature_types_exclude
        
        print(self.feature_types_exclude)    
        # a few shorthands for ignoring sets of features at a time
        if 'crosscorrs' in feature_types_exclude:
            feature_types_exclude.extend( [ff for ff in feature_types_all if 'crosscorrs' in ff])
        if 'autocorrs' in feature_types_exclude:
            feature_types_exclude.extend( [ff for ff in feature_types_all if 'autocorrs' in ff])
        if 'pixel' in feature_types_exclude:
            feature_types_exclude.extend(['pixel_stats'])

        self.feature_types_include  = [ff for ff in feature_types_all if not ff in feature_types_exclude]
        if len(self.feature_types_include)==0:
            raise ValueError('you have specified too many features to exclude, and now you have no features left! aborting.')
            
        feature_dims_include = [feature_type_dims[fi] for fi in range(len(feature_type_dims)) if not feature_types_all[fi] in feature_types_exclude]
        # how many features will be needed, in total?
        self.n_features_total = np.sum(feature_dims_include)
        
        # numbers that define which feature types are in which column
        self.feature_column_labels = np.squeeze(np.concatenate([fi*np.ones([1,feature_dims_include[fi]]) for fi in range(len(feature_dims_include))], axis=1).astype('int'))
        assert(np.size(self.feature_column_labels)==self.n_features_total)
        
        if self.group_all_hl_feats:
            # In this case pretend there are just two groups of features - the 'mean_magnitudes' which are first-level gabor-like
            # and all other features combined into a second group. Makes it simpler to do variance partition analysis.
            # if do_varpart=False, this does nothing.
            self.feature_column_labels[self.feature_column_labels != 1] = -1
            self.feature_column_labels[self.feature_column_labels==1] = 0
            self.feature_column_labels[self.feature_column_labels==-1] = 1
            self.feature_group_names = ['mean_magnitudes', 'all_other_texture_feats']
        else:
            self.feature_group_names = self.feature_types_include
            
    def get_partial_versions(self):
        
        if not hasattr(self, 'max_features'):
            raise RuntimeError('need to run init_for_fitting first')
            
        n_feature_types = len(self.feature_group_names)
        partial_version_names = ['full_model'] 
        masks = np.ones([1,self.n_features_total])
        
        if self.do_varpart:
            
            # "Partial versions" will be listed as: [full model, model w only first set of features, model w only second set, ...             
            partial_version_names += ['just_%s'%ff for ff in self.feature_group_names]
            masks2 = np.concatenate([np.expand_dims(np.array(self.feature_column_labels==ff).astype('int'), axis=0) for ff in np.arange(0,n_feature_types)], axis=0)
            masks = np.concatenate((masks, masks2), axis=0)
            
            if n_feature_types > 2:
                # if more than two types, also include models where we leave out first set of features, leave out second set of features...]
                partial_version_names += ['leave_out_%s'%ff for ff in self.feature_group_names]           
                masks3 = np.concatenate([np.expand_dims(np.array(self.feature_column_labels!=ff).astype('int'), axis=0) for ff in np.arange(0,n_feature_types)], axis=0)
                masks = np.concatenate((masks, masks3), axis=0)           
        
        # masks always goes [n partial versions x n total features]
        return masks, partial_version_names

    def get_maps(self, images):
    
        print('Running steerable pyramid feature extraction...')
        print('Images array shape is:')
        print(images.shape)
        t = time.time()
        fmaps = self.fmaps_fn(images, to_torch=False, device=self.device)        
        self.fmaps = fmaps
        elapsed =  time.time() - t
        print('time elapsed = %.5f'%elapsed)

    def clear_maps(self):
        
        print('Clearing steerable pyramid features from memory.')
        self.fmaps = None
        
    def forward(self, images, prf_params, prf_model_index, fitting_mode=True):
        
        if self.fmaps is None:
            self.get_maps(images)
        else:
            assert(images.shape[0]==self.fmaps[0][0].shape[0])
        
        if isinstance(prf_params, torch.Tensor):
            prf_params = torch_utils.get_value(prf_params)
        assert(np.size(prf_params)==3)
        prf_params = np.squeeze(prf_params)
        if isinstance(images, torch.Tensor):
            images = torch_utils.get_value(images)

        print('Computing higher order correlations...')
      
        t = time.time()
        pixel_stats, mean_magnitudes, mean_realparts, marginal_stats_lowpass_recons, variance_highpass_resid, \
            magnitude_feature_autocorrs, lowpass_recon_autocorrs, highpass_resid_autocorrs, \
            magnitude_within_scale_crosscorrs, real_within_scale_crosscorrs, magnitude_across_scale_crosscorrs, real_imag_across_scale_crosscorrs, \
            real_spatshift_within_scale_crosscorrs, real_spatshift_across_scale_crosscorrs =  \
                    get_higher_order_features(self.fmaps, images, prf_params, sample_batch_size=self.sample_batch_size, n_prf_sd_out=self.n_prf_sd_out, aperture=self.aperture, device=self.device)
        
        
        elapsed =  time.time() - t
        print('time elapsed = %.5f'%elapsed)

        all_feat = OrderedDict({'pixel_stats':pixel_stats, 'mean_magnitudes':mean_magnitudes, 'mean_realparts':mean_realparts, \
                                'marginal_stats_lowpass_recons':marginal_stats_lowpass_recons, 'variance_highpass_resid':variance_highpass_resid, \
            'magnitude_feature_autocorrs':magnitude_feature_autocorrs, 'lowpass_recon_autocorrs':lowpass_recon_autocorrs, 'highpass_resid_autocorrs':highpass_resid_autocorrs, \
            'magnitude_within_scale_crosscorrs':magnitude_within_scale_crosscorrs, 'real_within_scale_crosscorrs':real_within_scale_crosscorrs, \
            'magnitude_across_scale_crosscorrs':magnitude_across_scale_crosscorrs, 'real_imag_across_scale_crosscorrs':real_imag_across_scale_crosscorrs, \
            'real_spatshift_within_scale_crosscorrs':real_spatshift_within_scale_crosscorrs, 'real_spatshift_across_scale_crosscorrs':real_spatshift_across_scale_crosscorrs})

        feature_names_full = list(all_feat.keys())
        feature_names = [fname for fname in feature_names_full if fname in self.feature_types_include]
        self.feature_names = feature_names
        assert(feature_names==self.feature_types_include) # double check here that the order is correct
        
        for ff, feature_name in enumerate(feature_names):   
            assert(all_feat[feature_name] is not None)
            if ff==0:
                all_feat_concat = all_feat[feature_name]
            else:               
                all_feat_concat = torch.cat((all_feat_concat, all_feat[feature_name]), axis=1)

        assert(all_feat_concat.shape[1]==self.n_features_total)
        print('Final size of features concatenated is [%d x %d]'%(all_feat_concat.shape[0], all_feat_concat.shape[1]))
        print('Feature types included are:')
        print(feature_names)

        if torch.any(torch.isnan(all_feat_concat)):
            print('\nWARNING THERE ARE NANS IN FEATURES MATRIX\n')
        if torch.any(torch.sum(all_feat_concat, axis=0)==0):
            print('\nWARNING THERE ARE ZEROS IN FEATURES MATRIX\n')
            print('zeros for columns:')
            print(np.where(torch.sum(all_feat_concat, axis=0)==0))
            
        feature_inds_defined = np.ones((self.n_features_total,), dtype=bool)
        
        return all_feat_concat, feature_inds_defined
    

class steerable_pyramid_extractor(nn.Module):
    
    """
    Module that utilizes steerable pyramid ( https://pyrtools.readthedocs.io/en/latest/) to extract features.
    For a batch of input images, will return all the pyramid coefficients, as well as additional types of feature maps
    (i.e. partially reconstructed lowpass images at several frequency levels, upsampled feature maps).
    These are used by 'get_higher_order_features' to extract various textural features of the image.
    Adapted by MH from code in the library at:
    https://github.com/freeman-lab/metamers
    """
    
    def __init__(self, pyr_height=4, n_ori=8):
        
        super(steerable_pyramid_extractor, self).__init__()       
        self.n_ori = n_ori
        self.pyr_height = pyr_height # how many spatial frequencies?
        self.pyr = None
        
    def forward(self, image_batch, to_torch=False, device=None):
        
        batch_size = image_batch.shape[0]
        t  = time.time()
        for ii in range(batch_size):
            
            # Call the pyramid generation code here, get all features for this image.
            pyr = pt.pyramids.SteerablePyramidFreq(image_batch[ii,0,:,:], is_complex=True, height = self.pyr_height, order = self.n_ori-1)
            self.pyr = pyr # storing the most recently generated pyramid, in case we need its properties later
            
            if ii==0:  
                
                # Initialize all the feature maps we want to store...
                fmaps_complex = []
                fmaps_coarser_upsampled = []
               
                # Will generate several low-pass filtered representations of the image - will use these as additional feature maps for 
                # computing autocorrelations and marginal statistics.
                fmaps_lowpass_recon = []
                fmaps_lowpass_recon.append(np.zeros((batch_size,1,pyr.pyr_coeffs['residual_lowpass'].shape[0],pyr.pyr_coeffs['residual_lowpass'].shape[1])))
               
                # Feature maps will be listed from low SF to high SF. Each map stack is size [batch_size x orientations x height x width]
                sf_reverse  = self.pyr_height  # need to go backward because pyr comes out in the opposite order from what i want
                for sf in range(self.pyr_height):
                    sf_reverse -= 1
                    fmaps_complex.append(np.zeros((batch_size, self.n_ori,pyr.pyr_coeffs[sf_reverse,0].shape[0],\
                                                   pyr.pyr_coeffs[sf_reverse,0].shape[1]), dtype=complex))
                    
                    # Initialize "parent" representations for this level (map from the next coarsest resolution, upsampled to the finer scale)
                    # To be used for cross-scale comparisons.
                    if sf==0:
                        # this will be the lowpass residual (non-oriented).
                        fmaps_coarser_upsampled.append(np.zeros((batch_size, 1,pyr.pyr_coeffs[sf_reverse,0].shape[0],\
                                                   pyr.pyr_coeffs[sf_reverse,0].shape[1]), dtype=complex))                     
                    else:
                        fmaps_coarser_upsampled.append(np.zeros((batch_size, self.n_ori,pyr.pyr_coeffs[sf_reverse,0].shape[0],\
                                                   pyr.pyr_coeffs[sf_reverse,0].shape[1]), dtype=complex))     
                        
                    fmaps_lowpass_recon.append(np.zeros((batch_size,1,pyr.pyr_coeffs[sf_reverse,0].shape[0],\
                                                         pyr.pyr_coeffs[sf_reverse,0].shape[1])))

                fmaps_resid = []    
                fmaps_resid.append(np.zeros((batch_size,1,pyr.pyr_coeffs['residual_lowpass'].shape[0],pyr.pyr_coeffs['residual_lowpass'].shape[1])))
                fmaps_resid.append(np.zeros((batch_size,1,pyr.pyr_coeffs['residual_highpass'].shape[0],pyr.pyr_coeffs['residual_highpass'].shape[1])))

            # First get lowpass filtered representation of the image
            lowpass_recon = pyr.recon_pyr(levels='residual_lowpass', bands='all', twidth=1) 
            scale_by = pyr.pyr_size[(0,0)][0]/pyr.pyr_size['residual_lowpass'][0]
            lowpass_recon = texture_utils.shrink(lowpass_recon, scale_by)*scale_by**2
            fmaps_lowpass_recon[0][ii,0,:,:] = lowpass_recon
            
            # Get the "parent" for lowest SF level (upsample the residual lowpass)
#             print(pyr.pyr_coeffs['residual_lowpass'].shape)
            upsampled = texture_utils.expand(pyr.pyr_coeffs['residual_lowpass'], factor=2)/2**2
#             print(upsampled.shape)
            fmaps_coarser_upsampled[0][ii,0,:,:] = upsampled
            
            # Feature maps will be listed from low SF to high SF. Each map stack is size [batch_size x orientations x height x width]
            sf_reverse  = self.pyr_height # need to go backward because pyr comes out in the opposite order from what i want
            for sf in range(self.pyr_height):
                sf_reverse -= 1
                for oo in range(self.n_ori):     

                    # These are the main feature maps of the pyramid - one feature map per scale per orientation band.
                    # Complex number, can take the magnitude or real/imaginary part to simulate complex or simple cell-type responses.
                    fmaps_complex[sf][ii,oo,:,:] = pyr.pyr_coeffs[(sf_reverse,oo)]
                    
                    if sf<self.pyr_height-1:
                        # Store this as a "parent" representation, will be used for the next most fine SF level (i.e. sf+1)
                        upsampled = texture_utils.expand(pyr.pyr_coeffs[(sf_reverse,oo)], factor=2)/2**2
                        # Double the phase (angle of the complex number); note this doesn't affect the magnitude. 
                        phase_doubled = texture_utils.double_phase(upsampled)
                        fmaps_coarser_upsampled[sf+1][ii,oo,:,:] = phase_doubled
                   
                        
                # Get the bandpass filtered representation for this scale
                bandpass_image = np.real(pyr.recon_pyr(levels=sf_reverse, bands='all', twidth=1))
                scale_by = pyr.pyr_size[(0,0)][0]/pyr.pyr_size[(sf_reverse,0)][0]
                bandpass_image = texture_utils.shrink(bandpass_image, factor=scale_by)*scale_by**2
                
                # Add it onto the lowpass_recon (gets modified every loop iteration)
                lowpass_recon = texture_utils.expand(lowpass_recon, factor=2)/2**2
                lowpass_recon = lowpass_recon + bandpass_image
                fmaps_lowpass_recon[sf+1][ii,0,:,:] = lowpass_recon
            

            # Grab residual feature maps, the lowest and highest levels of the pyramid
            fmaps_resid[0][ii,0,:,:] = pyr.pyr_coeffs['residual_lowpass']
            fmaps_resid[1][ii,0,:,:] = pyr.pyr_coeffs['residual_highpass']
            
            
        elapsed = time.time() - t
#         print('time elapsed: %.5f s'%elapsed)

        if to_torch:            
            fmaps_complex = [torch.from_numpy(fm).to(device) for fm in fmaps_complex]            
            fmaps_resid = [torch_utils._to_torch(fm, device=device) for fm in fmaps_resid]
            fmaps_lowpass_recon = [torch_utils._to_torch(fm, device=device) for fm in fmaps_lowpass_recon]                      
            fmaps_coarser_upsampled = [torch.from_numpy(fm).to(device) for fm in fmaps_coarser_upsampled]

        return fmaps_complex, fmaps_resid, fmaps_lowpass_recon, fmaps_coarser_upsampled
    
   


def get_higher_order_features(fmaps, images, prf_params, sample_batch_size=20, n_prf_sd_out=2, aperture=1.0, device=None):

    """
    Compute higher order texture features for a batch of images.
    Input the module that defines steerable pyramid (i.e. 'steerable_pyramid_extractor'), and desired prf parameters.
    Returns arrays of each higher order feature.  
    Adapted by MH from code in the library at:
    https://github.com/freeman-lab/metamers
    """

    fmaps_complex_all, fmaps_resid_all, fmaps_lowpass_recon_all, fmaps_coarser_upsampled_all = fmaps
   
    n_trials = fmaps_complex_all[0].shape[0]
    x,y,sigma = prf_params

    n_sf = len(fmaps_complex_all)
    n_ori = fmaps_complex_all[0].shape[1]
        
    # all pairs of different orientation channels.
    ori_pairs = np.vstack([[[oo1, oo2] for oo2 in np.arange(oo1+1, n_ori)] for oo1 in range(n_ori) if oo1<n_ori-1])
    n_ori_pairs = np.shape(ori_pairs)[0]

    # mean, variance, skew, kurtosis, min, max
    pixel_stats = torch.zeros((n_trials,6), device=device)

    # Mean magnitude each scale/orientation, within the prf.
    mean_magnitudes = torch.zeros((n_trials, n_sf, n_ori), device=device)
    mean_realparts = torch.zeros((n_trials, n_sf, n_ori), device=device)

    # Store the skew and kurtosis of the lowpass reconstructions at each scale
    marginal_stats_lowpass_recons = torch.zeros((n_trials, n_sf+1, 2), device=device)

    # Variance of the highpass residual
    variance_highpass_resid = torch.zeros((n_trials, 1), device=device)

    # how many unique autocorrelation values will we get out for each feature map? These will be pre-defined, same for every pRF.
    # but different for different scales of feature maps.
    # note also that for bigger prfs, there will potentially be more pixels that contribute to the autocorrelation computation - 
    # but a fixed portion of the matrix is returned.
    autocorr_output_pix=np.array([3,3,5,7,7])
    n_autocorr_vals = ((autocorr_output_pix**2+1)/2).astype('int')
    max_autocorr_vals = np.max(n_autocorr_vals)
    
    # Spatial autocorrelation of the magnitude of spectral coefficients, within each scale and orientation.
    magnitude_feature_autocorrs = torch.zeros([n_trials, n_sf, n_ori, max_autocorr_vals], device=device) # this is ace in the matlab code

    # Spatial autocorrelation of the partially-reconstructed lowpass image representation at each scale
    lowpass_recon_autocorrs = torch.zeros([n_trials, n_sf+1, max_autocorr_vals], device=device) # this is acr in the matlab code

    # Spatial autocorrelation of the highpass residual
    highpass_resid_autocorrs = torch.zeros([n_trials, 1, max_autocorr_vals], device=device)

    # Within scale correlations of feature maps: compare feature map magnitudes for different orientations.
    magnitude_within_scale_crosscorrs = torch.zeros([n_trials, n_sf, n_ori_pairs], device=device) # this is C0 in the matlab code
    # Using the real parts.
    real_within_scale_crosscorrs = torch.zeros([n_trials, n_sf, n_ori_pairs], device=device) # this is Cr0 in the matlab code

    # Cross-scale correlations of feature maps: always comparing each scale to an up-sampled version of the scale coarser than it.
    magnitude_across_scale_crosscorrs = torch.zeros([n_trials, n_sf-1, n_ori, n_ori], device=device) # this is Cx0 in the matlab code

    # Cross-scale correlations, using the real and imaginary parts separately. The phase (angle) of the coarser map is doubled before computing these.
    real_imag_across_scale_crosscorrs = torch.zeros([n_trials, n_sf-1, 2, n_ori, n_ori], device=device) # this is Crx0 in the matlab code

    # These are comparisons with spatially shifted versions of the lowpass residual. Not sure we need this...
    n_spatshifts = 5;
    real_spatshift_within_scale_crosscorrs = torch.zeros([n_trials, 1, n_spatshifts, n_spatshifts], device=device)# this is Cr0 in the matlab code
    real_spatshift_across_scale_crosscorrs = torch.zeros([n_trials, 1, n_ori, n_spatshifts], device=device)  # this is Crx0 in the matlab code

    # Looping over batches of trials to compute everything of interest.
    bb=-1
    for batch_inds, batch_size_actual in numpy_utils.iterate_range(0, n_trials, sample_batch_size):
        bb=bb+1

        fmaps_complex = [torch.from_numpy(fmaps_complex_all[ii][batch_inds,:,:,:]).to(device) for ii in range(len(fmaps_complex_all))]
        fmaps_resid = [torch.from_numpy(fmaps_resid_all[ii][batch_inds,:,:,:]).float().to(device) for ii in range(len(fmaps_resid_all))]
        fmaps_lowpass_recon = [torch.from_numpy(fmaps_lowpass_recon_all[ii][batch_inds,:,:,:]).float().to(device) for ii in range(len(fmaps_lowpass_recon_all))]
        fmaps_coarser_upsampled = [torch.from_numpy(fmaps_coarser_upsampled_all[ii][batch_inds,:,:,:]).to(device) for ii in range(len(fmaps_coarser_upsampled_all))]

        if bb==0:
            npix_each_scale = [fmaps_complex_all[sc].shape[2] for sc in np.arange(n_sf-1,-1,-1)]
            npix_each_scale.append(fmaps_resid_all[0].shape[2])
            npix_each_scale.reverse()

        # First working with the finest scale (original image)
        n_pix = npix_each_scale[-1]      
        g = prf_utils.make_gaussian_mass_stack([x], [y], [sigma], n_pix=n_pix, size=aperture, dtype=np.float32)
        spatial_weights = g[2][0]
        patch_bbox_square = texture_utils.get_bbox_from_prf(prf_params, spatial_weights.shape, n_prf_sd_out, force_square=True, min_pix=autocorr_output_pix[-1])

        # Gather pixel-wise statistics here 
        wmean, wvar, wskew, wkurt = texture_utils.get_weighted_pixel_features(images[batch_inds], spatial_weights, device=device)
        pixel_stats[batch_inds,0] = torch.squeeze(wmean)
        pixel_stats[batch_inds,1] = torch.squeeze(wvar)
        pixel_stats[batch_inds,2] = torch.squeeze(wskew)
        pixel_stats[batch_inds,3] = torch.squeeze(wkurt)
        pixel_stats[batch_inds,4] = torch_utils._to_torch(np.squeeze(np.min(np.min(images[batch_inds], axis=3), axis=2)), device=device)
        pixel_stats[batch_inds,5] = torch_utils._to_torch(np.squeeze(np.max(np.max(images[batch_inds], axis=3), axis=2)), device=device)

        # Autocorrs of the highpass residual
        highpass_resid = fmaps_resid[1]
        auto_corr = texture_utils.weighted_auto_corr_2d(highpass_resid, spatial_weights, patch_bbox=patch_bbox_square, output_pix = autocorr_output_pix[-1], subtract_patch_mean = True, enforce_size=True, device=device)       
        highpass_resid_autocorrs[batch_inds,0,0:n_autocorr_vals[-1]] = torch.reshape(texture_utils.unique_autocorrs(auto_corr), [batch_size_actual, n_autocorr_vals[-1]])

        # Variance of the highpass residual
        m, wvar, s, k = texture_utils.get_weighted_pixel_features(highpass_resid, spatial_weights, device=device)
        variance_highpass_resid[batch_inds,0] = torch.squeeze(wvar)

        # Next work with the low-pass reconstruction (most coarse scale, smallest npix)
        n_pix = npix_each_scale[0]       
        g = prf_utils.make_gaussian_mass_stack([x], [y], [sigma], n_pix=n_pix, size=aperture, dtype=np.float32)
        spatial_weights = g[2][0]
        patch_bbox_square = texture_utils.get_bbox_from_prf(prf_params, spatial_weights.shape, n_prf_sd_out, force_square=True, min_pix=autocorr_output_pix[0])

        lowpass_rec = fmaps_lowpass_recon[0]

        # Marginal stats of low-pass reconstruction
        m, v, wskew, wkurt = texture_utils.get_weighted_pixel_features(lowpass_rec, spatial_weights, device=device)
        marginal_stats_lowpass_recons[batch_inds,0,0] = torch.squeeze(wskew)
        marginal_stats_lowpass_recons[batch_inds,0,1] = torch.squeeze(wkurt)

        # Autocorrs of low-pass reconstruction 
        auto_corr = texture_utils.weighted_auto_corr_2d(lowpass_rec, spatial_weights, patch_bbox=patch_bbox_square, output_pix = autocorr_output_pix[0], subtract_patch_mean = True, enforce_size=True, device=device)       
        lowpass_recon_autocorrs[batch_inds,0,0:n_autocorr_vals[0]] = torch.reshape(texture_utils.unique_autocorrs(auto_corr), [batch_size_actual, n_autocorr_vals[0]])

        # Looping over spatial frequency/scale
        # Loop goes low SF (smallest npix) to higher SF (largest npix)
        for ff in range(n_sf):
         
            # Scale specific things - get the prf at this resolution of interest    
            n_pix = npix_each_scale[ff+1]           
            g = prf_utils.make_gaussian_mass_stack([x], [y], [sigma], n_pix=n_pix, size=aperture, dtype=np.float32)
            spatial_weights = g[2][0]
            patch_bbox_square = texture_utils.get_bbox_from_prf(prf_params, spatial_weights.shape, n_prf_sd_out, force_square=True, min_pix=autocorr_output_pix[1+ff])

            # Get the low-pass reconstruction at this scale
            lowpass_summed = fmaps_lowpass_recon[ff+1]  # this is summed over this scale band and those below it
            m, v, wskew, wkurt = texture_utils.get_weighted_pixel_features(lowpass_summed, spatial_weights, device=device)
            marginal_stats_lowpass_recons[batch_inds,ff+1,0] = torch.squeeze(wskew)
            marginal_stats_lowpass_recons[batch_inds,ff+1,1] = torch.squeeze(wkurt)

            # Autocorrelations of low-pass reconstruction (at this scale)
            auto_corr = texture_utils.weighted_auto_corr_2d(lowpass_summed, spatial_weights, patch_bbox=patch_bbox_square, output_pix = autocorr_output_pix[ff+1], subtract_patch_mean = True, enforce_size=True, device=device)       
            lowpass_recon_autocorrs[batch_inds,ff+1,0:n_autocorr_vals[1+ff]] = torch.reshape(texture_utils.unique_autocorrs(auto_corr), [batch_size_actual, n_autocorr_vals[1+ff]])

            # Loop over orientation channels
            xx=-1
            for oo1 in range(n_ori):       

                # Magnitude of the complex coefficients; complex cell-like responses
                mag1 = torch.abs(fmaps_complex[ff][:,oo1,:,:].view([batch_size_actual,1,n_pix,n_pix])).float()

                # The mean magnitudes here are basically second-order spectral statistics, within the specified spatial region defined by weights
                wmean, v, s, k = texture_utils.get_weighted_pixel_features(mag1, spatial_weights/np.sum(spatial_weights), device=device)
                mean_magnitudes[batch_inds, ff, oo1] = torch.squeeze(wmean)
                
                mag1 = mag1 - torch.tile(torch.mean(torch.mean(mag1, axis=3, keepdim=True), axis=2, keepdim=True), [1,1,n_pix, n_pix])

                # Real parts of the complex coefficients; simple cell-like responses
                real1 = torch.real(fmaps_complex[ff][:,oo1,:,:].view([batch_size_actual,1,n_pix,n_pix])).float()    
                
                # Average of the real parts within the specified spatial region
                wmean, v, s, k = texture_utils.get_weighted_pixel_features(real1, spatial_weights/np.sum(spatial_weights), device=device)
                mean_realparts[batch_inds, ff, oo1] = torch.squeeze(wmean)

                # Complex cell autocorrelation (correlation w spatially shifted versions of itself)     
                auto_corr = texture_utils.weighted_auto_corr_2d(mag1, spatial_weights, patch_bbox=patch_bbox_square, output_pix = autocorr_output_pix[ff+1], subtract_patch_mean = True, enforce_size=True, device=device)       
                magnitude_feature_autocorrs[batch_inds,ff,oo1,0:n_autocorr_vals[1+ff]] = torch.reshape(texture_utils.unique_autocorrs(auto_corr), [batch_size_actual, n_autocorr_vals[1+ff]])

                # Within-scale correlations - comparing resp at orient==oo1 to responses at all other orientations, same scale.
                for oo2 in np.arange(oo1+1, n_ori):            
                    xx = xx+1 
                    assert(oo1==ori_pairs[xx,0] and oo2==ori_pairs[xx,1])

                    # Magnitude at the other orientation (oo2)
                    mag2 = torch.abs(fmaps_complex[ff][:,oo2,:,:].view([batch_size_actual,1,n_pix,n_pix])).float()      
                    mag2 = mag2 - torch.tile(torch.mean(torch.mean(mag2, axis=3, keepdim=True), axis=2, keepdim=True), [1,1,n_pix, n_pix])

                    # Correlate the magnitude feature maps for the two orientations, within scale
                    cross_corr = texture_utils.weighted_cross_corr_2d(mag1, mag2, spatial_weights, patch_bbox=None, subtract_patch_mean = True, device=device)/(n_pix*n_pix)
                    magnitude_within_scale_crosscorrs[batch_inds,ff,xx] = torch.squeeze(cross_corr);

                    # Real part at the other orientation (oo2)
                    real2 = torch.real(fmaps_complex[ff][:,oo2,:,:].view([batch_size_actual,1,n_pix,n_pix])).float()                     

                    # Correlate the real feature maps for the two orientations, within scale
                    cross_corr = texture_utils.weighted_cross_corr_2d(real1, real2, spatial_weights, patch_bbox=None, subtract_patch_mean = True, device=device)/(n_pix*n_pix)
                    real_within_scale_crosscorrs[batch_inds,ff,xx] = torch.squeeze(cross_corr);

                # Cross-scale correlations - for these we care about same ori to same ori, so looping over all orientations.
                # Going to compare coefficients at the current scale to those at a coarser scale (ff-1)
                # If we're at first scale (ff=0), then will use a different method.
                if ff>0:

                    for oo2 in range(n_ori):

                        # Get magnitude of coefficients for neighboring (coarser) scale                        
                        mag_coarser2 = torch.abs(fmaps_coarser_upsampled[ff][:,oo2,:,:].view([batch_size_actual,1,n_pix,n_pix])).float()
                        mag_coarser2 = mag_coarser2 - torch.tile(torch.mean(torch.mean(mag_coarser2, axis=3, keepdim=True), axis=2, keepdim=True), [1,1,n_pix, n_pix])

                        # Correlate this with the finer scale
                        cross_corr = texture_utils.weighted_cross_corr_2d(mag1, mag_coarser2, spatial_weights, patch_bbox=None, subtract_patch_mean = True, device=device)/(n_pix*n_pix)            
                        magnitude_across_scale_crosscorrs[batch_inds,ff-1,oo1,oo2] = torch.squeeze(cross_corr)

                        # Get the real and imaginary parts at coarser scale
                        real_coarser2 = torch.real(fmaps_coarser_upsampled[ff][:,oo2,:,:].view([batch_size_actual,1,n_pix,n_pix])).float()
                        imag_coarser2 = torch.imag(fmaps_coarser_upsampled[ff][:,oo2,:,:].view([batch_size_actual,1,n_pix,n_pix])).float()

                        # Correlate each of these with real part at finer scale
                        cross_corr = texture_utils.weighted_cross_corr_2d(real1, real_coarser2, spatial_weights, patch_bbox=None, subtract_patch_mean = True, device=device)/(n_pix*n_pix) 
                        real_imag_across_scale_crosscorrs[batch_inds,ff-1,0,oo1,oo2] = torch.squeeze(cross_corr)

                        cross_corr = texture_utils.weighted_cross_corr_2d(real1, imag_coarser2, spatial_weights, patch_bbox=None, subtract_patch_mean = True, device=device)/(n_pix*n_pix) 
                        real_imag_across_scale_crosscorrs[batch_inds,ff-1,1,oo1,oo2] = torch.squeeze(cross_corr)

                else:

                    # instead of different orientations for the "parent" level here, have spatially shifted versions.
                    real_coarser = torch.real(fmaps_coarser_upsampled[ff][:,0,:,:].view([batch_size_actual,1,n_pix,n_pix])).float()

                    shifts = [[0,0],[1,3],[-1,3],[1,2],[-1,2]]
                    for si1, shift1 in enumerate(shifts):

                        ss,dd = shift1
                        real_coarser_shifted1 = torch.roll(real_coarser, shifts=ss, dims=dd)               
                        # Real part at the finer scale compared to spatially shifted at the coarser scale
                        cross_corr = texture_utils.weighted_cross_corr_2d(real1, real_coarser_shifted1, spatial_weights, patch_bbox=None, subtract_patch_mean = True, device=device)/(n_pix*n_pix) 
                        real_spatshift_across_scale_crosscorrs[batch_inds,ff,oo1,si1] = torch.squeeze(cross_corr)

                        for si2 in np.arange(si1+1, n_spatshifts):

                            ss,dd = shifts[si2]
                            real_coarser_shifted2 = torch.roll(real_coarser, shifts=ss, dims=dd) 
                            # Real parts at same scale, comparing spatially shifted.
                            cross_corr = texture_utils.weighted_cross_corr_2d(real_coarser_shifted1, real_coarser_shifted2, spatial_weights, patch_bbox=None, subtract_patch_mean = True, device=device)/(n_pix*n_pix) 
                            real_spatshift_within_scale_crosscorrs[batch_inds,ff,si1,si2] = torch.squeeze(cross_corr)

            
    # Reshape everything to [ntrials x nfeatures]
    
    mean_magnitudes = torch.reshape(mean_magnitudes, [n_trials, -1])
    mean_realparts = torch.reshape(mean_realparts, [n_trials, -1])
    marginal_stats_lowpass_recons = torch.reshape(marginal_stats_lowpass_recons, [n_trials, -1])
    variance_highpass_resid =torch.reshape(variance_highpass_resid, [n_trials, -1])

    magnitude_feature_autocorrs = torch.reshape(magnitude_feature_autocorrs, [n_trials, -1])
    # take out the zero columns, which happen because of different size autocorr outputs.
    magnitude_feature_autocorrs = magnitude_feature_autocorrs[:,torch.sum(magnitude_feature_autocorrs, axis=0)!=0]
    assert(magnitude_feature_autocorrs.shape[1]==np.sum(n_autocorr_vals[1:]*n_ori))

    lowpass_recon_autocorrs = torch.reshape(lowpass_recon_autocorrs, [n_trials, -1])
    lowpass_recon_autocorrs = lowpass_recon_autocorrs[:,torch.sum(lowpass_recon_autocorrs, axis=0)!=0]
    assert(lowpass_recon_autocorrs.shape[1]==np.sum(n_autocorr_vals))

    highpass_resid_autocorrs = torch.reshape(highpass_resid_autocorrs, [n_trials, -1])

    magnitude_within_scale_crosscorrs = torch.reshape(magnitude_within_scale_crosscorrs, [n_trials, -1])
    real_within_scale_crosscorrs = torch.reshape(real_within_scale_crosscorrs, [n_trials, -1])
    magnitude_across_scale_crosscorrs = torch.reshape(magnitude_across_scale_crosscorrs, [n_trials, -1])
    real_imag_across_scale_crosscorrs = torch.reshape(real_imag_across_scale_crosscorrs, [n_trials, -1])
    real_spatshift_within_scale_crosscorrs = torch.reshape(real_spatshift_within_scale_crosscorrs, [n_trials, -1])
    
    real_spatshift_within_scale_crosscorrs = real_spatshift_within_scale_crosscorrs[:,torch.sum(real_spatshift_within_scale_crosscorrs, axis=0)!=0]
    assert(real_spatshift_within_scale_crosscorrs.shape[1]==np.sum(np.arange(1,n_spatshifts)))

    real_spatshift_across_scale_crosscorrs = torch.reshape(real_spatshift_across_scale_crosscorrs, [n_trials, -1])

        
    return pixel_stats, mean_magnitudes, mean_realparts, marginal_stats_lowpass_recons, variance_highpass_resid, \
            magnitude_feature_autocorrs, lowpass_recon_autocorrs, highpass_resid_autocorrs, \
            magnitude_within_scale_crosscorrs, real_within_scale_crosscorrs, magnitude_across_scale_crosscorrs, real_imag_across_scale_crosscorrs, \
            real_spatshift_within_scale_crosscorrs, real_spatshift_across_scale_crosscorrs




In [556]:
import numpy as np
import sys
import torch
import time
import torch.nn as nn
from utils import prf_utils, torch_utils, texture_utils, nms_utils
bdcn_path = '/user_data/mmhender/toolboxes/BDCN/'
sys.path.append(bdcn_path)
import bdcn

class bdcn_feature_extractor(nn.Module):
    
    def __init__(self, pretrained_model_file, device, aperture = 1.0, n_prf_sd_out = 2, \
                 batch_size=10, map_ind = -1, mult_patch_by_prf=True, downsample_factor = 1, do_nms = False, \
                 do_pca = True, min_pct_var = 99, max_pc_to_retain = 100):
        
        super(bdcn_feature_extractor, self).__init__()
        
        self.pretrained_model_file = pretrained_model_file
        self.device = device
        self.load_model_file()
          
        self.aperture = aperture
        self.n_prf_sd_out = n_prf_sd_out
        self.batch_size = batch_size
        self.map_ind = map_ind
        self.mult_patch_by_prf = mult_patch_by_prf
        if downsample_factor<1:
            raise ValueError('downsample factor must be >= 1')
        self.downsample_factor = downsample_factor
        self.do_nms = do_nms        
        self.fmaps = None
    
        self.do_pca = do_pca
        if self.do_pca:
            self.min_pct_var = min_pct_var
            self.max_pc_to_retain = max_pc_to_retain
        else:
            self.min_pct_var = None
            self.max_pc_to_retain = None  
            
        self.do_varpart=False # only one set of features in this model for now, not doing variance partition
  
    def load_model_file(self):
        
        model = bdcn.BDCN()
        model.load_state_dict(torch.load(self.pretrained_model_file))
        model = model.to(self.device)

        self.model = model
        
          
    def init_for_fitting(self, image_size, models, dtype):

        """
        Additional initialization operations which can only be done once we know image size and
        desired set of candidate prfs.
        """
        
        print('Initializing for fitting: finding the size of feature matrix for each candidate prf')
        n_prfs = len(models)
        downsampled_size = np.ceil(np.array(image_size)/self.downsample_factor).astype('int')
        n_feat_each_prf = np.zeros(shape=(n_prfs,),dtype=int)

        for mm in range(n_prfs):
            bbox = texture_utils.get_bbox_from_prf(models[mm], downsampled_size, n_prf_sd_out = self.n_prf_sd_out, \
                                                   min_pix=None, verbose=False, force_square=False)
            n_feat_each_prf[mm] =  (bbox[1] - bbox[0]) * (bbox[3] - bbox[2])           
        self.n_feat_each_prf = n_feat_each_prf;
        
        if self.do_pca:
            
            print('Initializing arrays for PCA params')
            # will need to save pca parameters to reproduce it during validation stage
            # max pc to retain is just to save space, otherwise the "pca_wts" variable becomes huge  
            self.max_features = self.max_pc_to_retain
            self.pca_wts = [np.zeros(shape=(self.max_pc_to_retain, n_feat_each_prf[mm]), dtype=dtype) for mm in range(n_prfs)] 
            self.pca_pre_mean = [np.zeros(shape=(n_feat_each_prf[mm],), dtype=dtype) for mm in range(n_prfs)]
            self.pct_var_expl = np.zeros(shape=(self.max_pc_to_retain, n_prfs), dtype=dtype)
            self.n_comp_needed = np.full(shape=(n_prfs), fill_value=-1, dtype=np.int)

        else:
            self.max_features = np.max(n_feat_each_prf)
            
        self.clear_maps()
 

    def get_partial_versions(self):

        if not hasattr(self, 'max_features'):
            raise RuntimeError('need to run init_for_fitting first')
           
        partial_version_names = ['full_model']
        masks = np.ones([1,self.max_features])

        return masks, partial_version_names
        
    def get_maps(self, images):
        
        print('Running BDCN contour feature extraction...')
        print('Images array shape is:')
        print(images.shape)
        t = time.time()
       
        maps_each_scale, names = get_bdcn_maps(self.model, images, self.batch_size, self.map_ind)
        maps = torch_utils._to_torch(maps_each_scale[0], device=self.device)

        if not self.downsample_factor==1:            
            orig_size = np.array(maps.shape[2:4])
            downsampled_size = np.ceil(orig_size/self.downsample_factor).astype('int')
            resized_maps = torch.nn.functional.interpolate(maps, \
                                                           size=(downsampled_size[0], downsampled_size[1]), \
                                                           mode = 'bilinear')
            maps = resized_maps
            print('Downsampled by factor of %.2f, new size is:'%self.downsample_factor)
            print(maps.shape)
                    
        maps = torch.sigmoid(maps)
        
        if self.do_nms:
            print('Applying non-maximum suppression to edge maps (can be slow...)')
            maps = nms_utils.apply_nms(maps)
        
        self.fmaps = maps
        
        print('Final array shape is:')
        print(maps.shape)
            
        elapsed =  time.time() - t
        print('time elapsed = %.5f'%elapsed)        
        
    def clear_maps(self):
        
        print('Clearing BDCN contour features from memory.')
        self.fmaps = None    
    
    def forward(self, images, prf_params, prf_model_index, fitting_mode = True):
        
        if self.fmaps is None:
            self.get_maps(images)
        else:
            assert(images.shape[0]==self.fmaps.shape[0])

        maps = self.fmaps  
        x,y,sigma = prf_params
        print('Getting features for pRF [x,y,sigma]:')
        print([x,y,sigma])
        n_pix = maps.shape[2]

         # Define the RF for this "model" version
        prf = torch_utils._to_torch(prf_utils.make_gaussian_mass(x, y, sigma, n_pix, size=self.aperture, \
                                  dtype=np.float32)[2], device=self.device)

        if self.mult_patch_by_prf:
            minval = torch.min(prf)
            maxval = torch.max(prf-minval)
            prf_scaled = (prf - minval)/maxval
            # Multiply the feature map by gaussian pRF weights, before cropping
            maps = maps * prf_scaled
        
        # Crop the patch +/- n SD away from center
        bbox = texture_utils.get_bbox_from_prf(prf_params, prf.shape, self.n_prf_sd_out, min_pix=None, verbose=False, force_square=False)
        print('bbox to crop is:')
        print(bbox)
        maps_cropped = maps[:,:,bbox[0]:bbox[1], bbox[2]:bbox[3]]
        
        print('[min max] of first image patch is:')
        print([torch.min(maps_cropped[0,0,:,:]), torch.max(maps_cropped[0,0,:,:])])
        
        # return [ntrials x nfeatures]
        # Note this reshaping goes in "C" style order by default
        features = torch.reshape(maps_cropped, [maps_cropped.shape[0], np.prod(maps_cropped.shape[1:])])
                
        if self.do_pca:    
            features = self.reduce_pca(features, prf_model_index, fitting_mode)

        feature_inds_defined = np.zeros((self.max_features,), dtype=bool)
        feature_inds_defined[0:features.shape[1]] = 1
            
        return features, feature_inds_defined
     
        
    def reduce_pca(self, features, prf_model_index, fitting_mode=True):
        
        if torch.is_tensor(features):
            features = features.detach().cpu().numpy()
            was_tensor=True
        else:
            was_tensor=False
            
        n_trials = features.shape[0]
        n_features_actual = features.shape[1]
        assert(n_features_actual == self.n_feat_each_prf[prf_model_index])
        print('Preparing for PCA: original dims of features:')
        print(features.shape)
        
        if fitting_mode:
            
            # Going to perform pca on the raw features
            # First make sure it hasn't been done yet!
            assert(self.n_comp_needed[prf_model_index]==-1) 
            print('Running PCA...')
            pca = decomposition.PCA(n_components = np.min([np.min([self.max_pc_to_retain, n_features_actual]), n_trials]), copy=False)
            # Perform PCA to decorrelate feats and reduce dimensionality
            scores = pca.fit_transform(features)           
            features = None            
            wts = pca.components_
            ev = pca.explained_variance_
            ev = ev/np.sum(ev)*100
            # wts/components goes [ncomponents x nfeatures]. 
            # nfeatures is always actual number of raw features
            # ncomponents is min(ntrials, nfeatures)
            # to save space, only going to save up to some max number of components.
            n_components_actual = np.min([wts.shape[0], self.max_pc_to_retain])
            # save a record of the transformation to be used for validating model
            self.pca_wts[prf_model_index][0:n_components_actual,0:n_features_actual] = wts[0:n_components_actual,:] 
            # mean of each feature, nfeatures long - needed to reproduce transformation
            self.pca_pre_mean[prf_model_index][0:n_features_actual] = pca.mean_ 
            # max len of ev is the number of components
            self.pct_var_expl[0:n_components_actual,prf_model_index] = ev[0:n_components_actual]  
            n_components_reduced = int(np.where(np.cumsum(ev)>self.min_pct_var)[0][0] if np.any(np.cumsum(ev)>self.min_pct_var) else len(ev))
            self.n_comp_needed[prf_model_index] = n_components_reduced
            print('Retaining %d components to expl %d pct var'%(n_components_reduced, self.min_pct_var))
            assert(n_components_reduced<=self.max_pc_to_retain)            
            features_reduced = scores[:,0:n_components_reduced]
           
        else:
            
            # This is a validation pass, going to use the pca pars that were computed on training set
            # Make sure it has been done already!
            assert(self.n_comp_needed[prf_model_index]!=-1)
            print('Applying pre-computed PCA matrix...')
            # Apply the PCA transformation, just as it was done during training
            features_submean = features - np.tile(np.expand_dims(self.pca_pre_mean[prf_model_index][0:n_features_actual], axis=0), [n_trials, 1])
            features_reduced = features_submean @ np.transpose(self.pca_wts[prf_model_index][0:self.n_comp_needed[prf_model_index],0:n_features_actual])               
                       
        features = None
        
        if was_tensor:
            features_reduced = torch.tensor(features_reduced).to(self.device)
        
        return features_reduced
    
    

def get_bdcn_maps(model, images, batch_size=10, map_inds=None):
            
    device = list(model.parameters())[0].device
 
    if map_inds is not None:
        if np.isscalar(map_inds):
            map_inds = [map_inds]
    else:
        map_inds = np.arange(0,11)
        
    n_images = images.shape[0]
    n_batches = int(np.ceil(n_images/batch_size))

    for bb in range(n_batches):

        batch_inds = np.arange(batch_size * bb, np.min([batch_size * (bb+1), n_images]))
        image_batch = images[batch_inds,:,:,:]
        
        out = model(prep_for_bdcn(image_batch, device))    
        out = [oo.detach().cpu().numpy() for oo in out]

        p1_1, p2_1, p3_1, p4_1, p5_1, p1_2, p2_2, p3_2, p4_2, p5_2, fuse = out
        
        # undoing the sums they do at end of forward pass, to get out the raw feature maps at each scale
        # p1_1 = s1
        # p2_1 = s2 + o1
        # p3_1 = s3 + o2 + o1
        # p4_1 = s4 + o3 + o2 + o1
        # p5_1 = s5 + o4 + o3 + o2 + o1

        # p1_2 = s11 + o21 + o31 + o41 + o51
        # p2_2 = s21 + o31 + o41 + o51
        # p3_2 = s31 + o41 + o51
        # p4_2 = s41 + o51
        # p5_2 = s51

        s1_1 = p1_1
        s2_1 = p2_1 - s1_1
        s3_1 = p3_1 - s1_1 - s2_1
        s4_1 = p4_1 - s1_1 - s2_1 - s3_1
        s5_1 = p5_1 - s1_1 - s2_1 - s3_1 - s4_1

        s5_2 = p5_2
        s4_2 = p4_2 - s5_2
        s3_2 = p3_2 - s5_2 - s4_2
        s2_2 = p2_2 - s5_2 - s4_2 - s3_2
        s1_2 = p1_2 - s5_2 - s4_2 - s3_2 - s2_2

        maps_each_scale_this_batch = [s1_1, s2_1, s3_1, s4_1, s5_1, s1_2, s2_2, s3_2, s4_2, s5_2, fuse]

        if bb==0:
            maps_each_scale = [maps_each_scale_this_batch[mi] for mi in map_inds]
        else:
            for ii, mi in enumerate(map_inds):                    
                maps_each_scale[ii] = np.concatenate((maps_each_scale[ii], maps_each_scale_this_batch[mi]), axis=0)
            

    names1 = ['s2d_%d'%(ii+1) for ii in range(5)]
    names2 = ['d2s_%d'%(ii+1) for ii in range(5)]
    names = list(np.concatenate([names1, names2, ['BDCN: fused']]))

    return maps_each_scale, names


def prep_for_bdcn(image_data, device):
    
    if image_data.shape[1]==1:
        image_data = np.tile(image_data, [1,3,1,1])
    else:
        # RGB to BGR
        image_data = image_data[:,-1::,:,:]   
    mean_bgr = np.expand_dims(np.array([104.00699, 116.66877, 122.67892]), [0,2,3])
    dat = image_data * 255 - mean_bgr
    dat = torch.tensor(dat, dtype=torch.float32).to(device)

    return dat



In [775]:
import sys
import os
import struct
import time
import copy
import numpy as np
import h5py
from tqdm import tqdm
import pickle
import math
import sklearn
from sklearn import decomposition

import torch
import torch.nn as nn
import torch.nn.init as I
import torch.nn.functional as F
import torch.optim as optim

from utils import numpy_utils, torch_utils, texture_utils

def _cofactor_fn_cpu(_x, lambdas):
    '''
    Generating a matrix needed to solve ridge regression model for each lambda value.
    Ridge regression (Tikhonov) solution is :
    w = (X^T*X + I*lambda)^-1 * X^T * Y
    This func will return (X^T*X + I*lambda)^-1 * X^T. 
    So once we have that, can just multiply by training data (Y) to get weights.
    returned size is [nLambdas x nFeatures x nTrials]
    This version makes sure that the torch inverse operation is done on the cpu, and in floating point-64 precision.
    Otherwise get bad results for small lambda values. This seems to be a torch-specific bug.
    
    '''
    device_orig = _x.device
    type_orig = _x.dtype
    # switch to this specific format which works with inverse
    _x = _x.to('cpu').to(torch.float64)
    _f = torch.stack([(torch.mm(torch.t(_x), _x) + torch.eye(_x.size()[1], device='cpu', dtype=torch.float64) * l).inverse() for l in lambdas], axis=0) 
    
    # [#lambdas, #feature, #feature] 
    cof = torch.tensordot(_f, _x, dims=[[2],[1]]) # [#lambdas, #feature, #sample]
    
    # put back to whatever way it was before, so that we can continue with other operations as usual
    return cof.to(device_orig).to(type_orig)



def _loss_fn(_cofactor, _vtrn, _xout, _vout):
    '''
    Calculate loss given "cofactor" from cofactor_fn, training data, held-out design matrix, held out data.
    returns weights (betas) based on equation
    w = (X^T*X + I*lambda)^-1 * X^T * Y
    also returns loss for these weights w the held out data. SSE is loss func here.
    '''

    _beta = torch.tensordot(_cofactor, _vtrn, dims=[[2], [0]]) # [#lambdas, #feature, #voxel]
    _pred = torch.tensordot(_xout, _beta, dims=[[1],[1]]) # [#samples, #lambdas, #voxels]
    _loss = torch.sum(torch.pow(_vout[:,None,:] - _pred, 2), dim=0) # [#lambdas, #voxels]
    return _beta, _loss


def fit_fwrf_model(images, voxel_data, _feature_extractor, prf_models, lambdas, \
                   zscore=False, add_bias=False, voxel_batch_size=100, holdout_size=100, \
                       shuffle=True, shuff_rnd_seed=0, device=None, do_varpart=False, debug=False):
    
    """
    Solve for encoding model weights using ridge regression.
    Inputs:
        images: the training images, [n_trials x 1 x height x width]
        voxel_data: the training voxel data, [n_trials x n_voxels]
        _feature_extractor_fn: module that maps from images to model features
        prf_models: the list of possible pRFs to test, columns are [x, y, sigma]
        lambdas: ridge lambda parameters to test
        zscore: want to zscore each column of feature matrix before fitting?
        add_bias: add a column of ones to feature matrix, for an additive bias?
        voxel_batch_size: how many voxels to use at a time for model fitting
        holdout_size: how many training trials to hold out for computing loss/lambda selection?
        shuffle: do we shuffle training data order before holding trials out?      
        shuff_rnd_seed: if we do shuffle training data (shuffle=True), what random seed to use? if zero, choose a new random seed in this code.
        device: what device to use? cpu/cuda
        do_varpart: perform variance partition by leaving out subsets of features at a time?
        debug: want to run a shortened version of this, to test it?
    Outputs:
        best_losses: loss value for each voxel (with best pRF and best lambda), eval on held out set
        best_lambdas: best lambda for each voxel (chosen based on loss w held out set)
        best_params: 
            [0] best pRF for each voxel [x,y,sigma]
            [1] best weights for each voxel/feature
            [2] if add_bias=True, best bias value for each voxel
            [3] if zscore=True, the mean of each feature before z-score
            [4] if zscore=True, the std of each feature before z-score
            [5] index of the best pRF for each voxel (i.e. index of row in "prf_models")
        
    """

    dtype = images.dtype.type
    if device is None:
        device=torch.device('cpu:0')

    print ('dtype = %s' % dtype)
    print ('device = %s' % device)

    n_trials = len(images)
    n_prfs = len(prf_models)
    n_voxels = voxel_data.shape[1]   

    # Get train/holdout splits.
    # Held-out data here is used for lamdba selection.
    # This is the inner part of nested cross-validation; there is another portion of data ('val') which never enters this function.
    trn_size = n_trials - holdout_size
    assert trn_size>0, 'Training size needs to be greater than zero'
    print ('trn_size = %d (%.1f%%)' % (trn_size, float(trn_size)*100/len(voxel_data)))
    order = np.arange(len(voxel_data), dtype=int)
    if shuffle:
        if shuff_rnd_seed==0:
            print('Computing a new random seed')
            shuff_rnd_seed = int(time.strftime('%M%H%d', time.localtime()))
        print('Seeding random number generator: seed is %d'%shuff_rnd_seed)
        np.random.seed(shuff_rnd_seed)
        np.random.shuffle(order)
    images = images[order]
    voxel_data = voxel_data[order]  
    trn_data = voxel_data[:trn_size]
    out_data = voxel_data[trn_size:]

    
    # Here is where any model-specific additional initialization steps are done
    # Includes initializing pca params arrays, if doing pca
    _feature_extractor.init_for_fitting(images.shape[2:4], prf_models, dtype)
    max_features = _feature_extractor.max_features

    # Decide whether to do any "partial" versions of the models (leaving out subsets of features)
    # Purpose is for variance partition
    masks, partial_version_names = _feature_extractor.get_partial_versions()
    n_partial_versions = len(partial_version_names) # will be one if skipping varpart
    if add_bias:
        masks = np.concatenate([masks, np.ones([masks.shape[0],1])], axis=1) # always include intercept 
    masks = np.transpose(masks)
    # masks is [n_features_total (including intercept) x n_partial_versions]

    # Initialize arrays to store model fitting params
    best_w_params = np.zeros(shape=(n_voxels, max_features ,n_partial_versions), dtype=dtype)
    best_prf_models = np.full(shape=(n_voxels,n_partial_versions), fill_value=-1, dtype=int)   
    best_lambdas = np.full(shape=(n_voxels,n_partial_versions), fill_value=-1, dtype=int)
    best_losses = np.full(fill_value=np.inf, shape=(n_voxels,n_partial_versions), dtype=dtype)

    # Additional params that are optional
    if add_bias:
        best_w_params = np.concatenate([best_w_params, np.ones(shape=(n_voxels,1,n_partial_versions), dtype=dtype)], axis=1)

    if zscore:
        features_mean = np.zeros(shape=(n_voxels, max_features), dtype=dtype)
        features_std  = np.zeros(shape=(n_voxels, max_features), dtype=dtype)
    else:
        features_mean = None
        features_std = None

    start_time = time.time()
    vox_loop_time = 0

    print ('---------------------------------------\n')
    
    with torch.no_grad():
        
        # Looping over prf_models (here prf_models are different spatial RF definitions)
        for m,(x,y,sigma) in enumerate(prf_models):
            if debug and m>1:
                break
                
            print('\nGetting features for prf %d: [x,y,sigma] is [%.2f %.2f %.4f]'%(m, prf_models[m,0],  prf_models[m,1],  prf_models[m,2]))

            t = time.time()            

            # Get features for the desired pRF, across all trn set image            
            features, feature_inds_defined = _feature_extractor(images, (x,y,sigma), m, fitting_mode=True)
            features = features.detach().cpu().numpy() 
            
            elapsed = time.time() - t

            n_features_actual = features.shape[1]
            
            if zscore:  
                features_m = np.mean(features, axis=0, keepdims=True) #[:trn_size]
                features_s = np.std(features, axis=0, keepdims=True) + 1e-6          
                features -= features_m
                features /= features_s    

            if add_bias:
                features = np.concatenate([features, np.ones(shape=(len(features), 1), dtype=dtype)], axis=1)
                feature_inds_defined = np.concatenate((feature_inds_defined, [True]), axis=0)
                
            trn_features = features[:trn_size,:]
            out_features = features[trn_size:,:]
            
            
            # Going to keep track of whether current prf is better than running best, for each voxel.
            # This is for the full model only.
            # Will use this to make sure for each partial model, we end up saving the params for the prf that was best w full model.
            full_model_improved = np.zeros((n_voxels,),dtype=bool)

            # Looping over versions of model w different features set to zero (variance partition)
            for pp in range(n_partial_versions):

                print('\nFitting version %d of %d: %s, '%(pp, n_partial_versions, partial_version_names[pp]))

                nonzero_inds_full = np.logical_and(masks[:,pp], feature_inds_defined)             
               
                # Send matrices to gpu
                nonzero_inds_short = masks[feature_inds_defined,pp]==1
                _xtrn = torch_utils._to_torch(trn_features[:, nonzero_inds_short], device=device)
                _xout = torch_utils._to_torch(out_features[:, nonzero_inds_short], device=device)   

                # Do part of the matrix math involved in ridge regression optimization out of the loop, 
                # because this part will be same for all the voxels.
                _cof = _cofactor_fn_cpu(_xtrn, lambdas = lambdas) 

                # Now looping over batches of voxels (only reason is because can't store all in memory at same time)
                vox_start = time.time()
                vi=-1
                for rv,lv in numpy_utils.iterate_range(0, n_voxels, voxel_batch_size):
                    vi=vi+1
                    sys.stdout.write('\rfitting model %4d of %-4d, voxels [%6d:%-6d] of %d' % (m, n_prfs, rv[0], rv[-1], n_voxels))

                    # Send matrices to gpu
                    _vtrn = torch_utils._to_torch(trn_data[:,rv], device=device)
                    _vout = torch_utils._to_torch(out_data[:,rv], device=device)

                    # Here is where optimization happens - relatively simple matrix math inside loss fn.
                    _betas, _loss = _loss_fn(_cof, _vtrn, _xout, _vout) #   [#lambda, #feature, #voxel, ], [#lambda, #voxel]
                    # Now have a set of weights (in betas) and a loss value for every voxel and every lambda. 
                    # goal is then to choose for each voxel, what is the best lambda and what weights went with that lambda.

                    # choose best lambda value and the loss that went with it.
                    _loss_values, _lambda_index = torch.min(_loss, dim=0)
                    loss_values, lambda_index = torch_utils.get_value(_loss_values), torch_utils.get_value(_lambda_index)
                    betas = torch_utils.get_value(_betas)


                    if pp==0:

                        # comparing this loss to the other prf_models for each voxel (e.g. the other RF position/sizes)
                        assert(partial_version_names[pp]=='full_model' or partial_version_names[pp]=='full_combined_model')               
                        imp = loss_values<best_losses[rv,pp]
                        full_model_improved[rv] = imp

                    else:

                        # for the partial models we don't actually care which was best for the partial model itself,
                        # just care what was best for the full model
                        imp = full_model_improved[rv]


                    if np.sum(imp)>0:

                        # for whichever voxels had improvement relative to previous prf_models, save parameters now
                        # this means we won't have to save all params for all prf_models, just best.
                        arv = np.array(rv)[imp]

                        lambda_inds = lambda_index[imp]
                        best_lambdas[arv,pp] = lambda_inds
                        best_losses[arv,pp] = loss_values[imp]
                        best_prf_models[arv,pp] = m
                        if zscore:
                            
                            fmean_tmp = copy.deepcopy(features_mean[arv,:])
                            fstd_tmp = copy.deepcopy(features_std[arv,:])
                            fmean_tmp[:,nonzero_inds_full[0:-1]] = features_m[0,nonzero_inds_short[0:-1]] # broadcast over updated voxels
                            fmean_tmp[:,~nonzero_inds_full[0:-1]] = 0.0
                            fstd_tmp[:,nonzero_inds_full[0:-1]] = features_s[0,nonzero_inds_short[0:-1]] # broadcast over updated voxels
                            fstd_tmp[:,~nonzero_inds_full[0:-1]] = 0.0
                            features_mean[arv,:] = fmean_tmp
                            features_std[arv,:] = fstd_tmp
                            
                        # taking the weights associated with the best lambda value
                        # remember that they won't fill entire matrix, rest of values stay at zero
                        best_w_tmp = copy.deepcopy(best_w_params[arv,:,pp])
                        best_w_tmp[:,nonzero_inds_full] = numpy_utils.select_along_axis(betas[:,:,imp], lambda_inds, run_axis=2, choice_axis=0).T
                        best_w_tmp[:,~nonzero_inds_full] = 0.0 # make sure to fill zeros here

#                         # bias is always last value, even if zeros for some other features
#                         if add_bias:
#                             best_w_tmp[:,-1] = numpy_utils.select_along_axis(betas[:,-1,imp], lambda_inds, run_axis=1, choice_axis=0).T

                        best_w_params[arv,:,pp] = best_w_tmp
                
                vox_loop_time += (time.time() - vox_start)
                elapsed = (time.time() - vox_start)
                sys.stdout.flush()

    # Print information about how fitting went...
    total_time = time.time() - start_time
    inv_time = total_time - vox_loop_time
    return_params = [best_w_params[:,0:max_features,:],]
    if add_bias:
        return_params += [best_w_params[:,-1,:],]
    else: 
        return_params += [None,]
    print ('\n---------------------------------------')
    print ('total time = %fs' % total_time)
    print ('total throughput = %fs/voxel' % (total_time / n_voxels))
    print ('voxel throughput = %fs/voxel' % (vox_loop_time / n_voxels))
    print ('setup throughput = %fs/model' % (inv_time / n_prfs))
    
    # This step clears the big feature maps for training data from feature extractor (no longer needed)
    _feature_extractor.clear_maps()
    
    best_params = [prf_models[best_prf_models],]+return_params+[features_mean, features_std]+[best_prf_models]
    sys.stdout.flush()

    return best_losses, best_lambdas, best_params


In [776]:
images = trn_stim_data
voxel_data = trn_voxel_data
prf_models = models
zscore_features = True
add_bias = True
voxel_batch_size=100
holdout_size=100
shuffle=True
shuff_rnd_seed = 842348
do_varpart=True
debug=True

best_losses, best_lambdas, best_params = fit_fwrf_model(images, voxel_data, _feature_extractor, prf_models, lambdas, \
                   zscore=zscore_features, add_bias=add_bias, voxel_batch_size=voxel_batch_size, holdout_size=holdout_size, \
                       shuffle=shuffle, shuff_rnd_seed=shuff_rnd_seed, device=device, do_varpart=do_varpart, debug=debug)

dtype = <class 'numpy.float32'>
device = cuda:0
trn_size = 588 (85.5%)
Seeding random number generator: seed is 842348
Initializing for fitting: finding the size of feature matrix for each candidate prf
Initializing arrays for PCA params


/home/mmhender/myenv/lib/python3.7/site-packages/ipykernel_launcher.py:80: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


Clearing BDCN contour features from memory.
Clearing steerable pyramid features from memory.
---------------------------------------


Getting features for prf 0: [x,y,sigma] is [-0.55 -0.55 0.0400]
Running BDCN contour feature extraction...
Images array shape is:
(688, 1, 240, 240)
Final array shape is:
torch.Size([688, 1, 240, 240])
time elapsed = 7.65913
Getting features for pRF [x,y,sigma]:
[-0.55, -0.55, 0.03999999910593033]
bbox to crop is:
[232, 240, 0, 9]
[min max] of first image patch is:
[tensor(0.0173, device='cuda:0'), tensor(0.3324, device='cuda:0')]
Preparing for PCA: original dims of features:
(688, 72)
Running PCA...
Retaining 9 components to expl 99 pct var
Running steerable pyramid feature extraction...
Images array shape is:
(688, 1, 240, 240)
time elapsed = 136.86514
Computing higher order correlations...
time elapsed = 7.37654
Final size of features concatenated is [688 x 641]
Feature types included are:
['pixel_stats', 'mean_magnitudes', 'mean_realparts', 'margina

In [777]:
best_params[3].shape

(14913, 1041)

In [781]:
best_params[4][0,0:20]

array([0.5106623 , 0.17976591, 0.13310748, 0.08246855, 0.06214282,
       0.0578819 , 0.05178565, 0.03472991, 0.0284207 , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ],
      dtype=float32)

In [780]:
best_params[1][0,0:20,0]

array([ 5.9986999e-04,  3.4265858e-04, -6.4414664e-05,  2.8409972e-04,
       -4.5182815e-04,  7.2751415e-04,  3.7275028e-04,  2.1712780e-03,
        7.2326441e-04,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
        0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
        0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
      dtype=float32)

In [846]:
import sys
import os
import struct
import time
import numpy as np
import tqdm
import copy

import torch

from utils import numpy_utils, torch_utils


def get_r2(actual,predicted):
  
    """
    This computes the coefficient of determination (R2).
    For OLS, this is a good measure of variance explained. 
    Not necessarily true for ridge regression - can use signed correlation coefficient^2 instead.
    With OLS & when train/test sets are identical, R2 = correlation coefficient^2.
    """
    
    # calculate r2 for this fit.
    ssres = np.sum(np.power((predicted - actual),2));
    sstot = np.sum(np.power((actual - np.mean(actual)),2));
    r2 = 1-(ssres/sstot)
    
    return r2
 

In [982]:
images = val_stim_data
voxel_data = val_voxel_data

params = best_params
dtype = images.dtype.type
device = _texture_fn.device

n_trials, n_voxels = len(images), len(params[0])
n_prfs = prf_models.shape[0]
n_features = params[1].shape[1]  
n_voxels = np.shape(voxel_data)[1]

best_models, weights, bias, features_mt, features_st, best_model_inds = params
masks, partial_version_names = _feature_extractor.get_partial_versions()
masks = np.transpose(masks)    
n_features_max = _feature_extractor.max_features
n_partial_versions = len(partial_version_names)

# val_cc is the correlation coefficient bw real and predicted responses across trials, for each voxel.
val_cc  = np.zeros(shape=(n_voxels, n_partial_versions), dtype=dtype)
val_r2 = np.zeros(shape=(n_voxels, n_partial_versions), dtype=dtype)

pred_models = np.full(fill_value=0, shape=(n_trials, n_features_max, n_prfs), dtype=dtype)
feature_inds_defined_each_prf = np.full(fill_value=0, shape=(n_features_max, n_prfs), dtype=bool)

start_time = time.time()  

In [983]:

_feature_extractor.clear_maps()

for mm in range(n_prfs):
    if mm>1 and debug:
        break
    print('Getting features for prf %d: [x,y,sigma] is [%.2f %.2f %.4f]'%(mm, prf_models[mm,0],  prf_models[mm,1],  prf_models[mm,2] ))
    all_feat_concat, feature_inds_defined = _feature_extractor(images, prf_models[mm,:], mm, fitting_mode=False)

    pred_models[:,feature_inds_defined,mm] = torch_utils.get_value(all_feat_concat)
    feature_inds_defined_each_prf[:,mm] = feature_inds_defined

_feature_extractor.clear_maps()



Clearing BDCN contour features from memory.
Clearing steerable pyramid features from memory.
Getting features for prf 0: [x,y,sigma] is [-0.55 -0.55 0.0400]
Running BDCN contour feature extraction...
Images array shape is:
(62, 1, 240, 240)
Final array shape is:
torch.Size([62, 1, 240, 240])
time elapsed = 0.64549
Getting features for pRF [x,y,sigma]:
[-0.55, -0.55, 0.03999999910593033]
bbox to crop is:
[232, 240, 0, 9]
[min max] of first image patch is:
[tensor(0.0058, device='cuda:0'), tensor(0.3294, device='cuda:0')]
Preparing for PCA: original dims of features:
(62, 72)
Applying pre-computed PCA matrix...
Running steerable pyramid feature extraction...
Images array shape is:
(62, 1, 240, 240)
time elapsed = 10.96169
Computing higher order correlations...
time elapsed = 0.48873
Final size of features concatenated is [62 x 641]
Feature types included are:
['pixel_stats', 'mean_magnitudes', 'mean_realparts', 'marginal_stats_lowpass_recons', 'variance_highpass_resid', 'magnitude_featur

In [961]:
np.sum(features_to_use)

1041

In [962]:
features_to_use.shape

(1041,)

In [964]:
np.sum(features_to_use & feature_inds_defined_each_prf[:,0])

650

In [965]:
is_defined = feature_inds_defined_each_prf[features_to_use,0]
np.sum(is_defined)

650

In [955]:
feature_inds_defined_each_prf.shape

(1041, 875)

In [828]:
sum(feature_inds_defined_each_prf[:,0])

650

In [830]:
features_full = pred_models[:,:,best_model_inds[rv,pp]]
# Note there may be some zeros in this matrix, if we used fewer than the max number of features.
# But they are zero in weight matrix too, so turns out ok.

_weights = torch_utils._to_torch(weights[rv,:,pp], device=device)   

_bias = torch_utils._to_torch(bias[rv,pp], device=device)


In [831]:
features_full.shape

(62, 1041, 100)

In [832]:
_weights.shape

torch.Size([100, 1041])

In [833]:

if features_mt is not None:
    _features_m = torch_utils._to_torch(features_mt[rv,:], device=device)
    _features_m = _features_m[:,feature_inds_defined_each_prf[:,pp]]
if features_st is not None:
    _features_s = torch_utils._to_torch(features_st[rv,:], device=device)
    _features_s = _features_s[:,feature_inds_defined_each_prf[:,pp]]


In [984]:

rv = range(100)
lv=100
pp=3
# vv=-1
# ## Looping over voxels here in batches, will eventually go through all.
# for rv, lv in numpy_utils.iterate_range(0, n_voxels, voxel_batch_size):
#     vv=vv+1
#     print('Getting predictions for voxels [%d-%d] of %d'%(rv[0],rv[-1],n_voxels))

#     if vv>1 and debug:
#         break

#     # Looping over versions of model w different features set to zero (variance partition)
#     for pp in range(n_partial_versions):

#         nonzero_inds_full = np.logical_and(masks[:,pp], feature_inds_defined[:,mm])             
is_defined = feature_inds_defined_each_prf[features_to_use,mm]

print('\nEvaluating version %d of %d: %s'%(pp, n_partial_versions, partial_version_names[pp]))

features_to_use = masks[:,pp]==1
print('Includes %d features'%np.sum(features_to_use))

# [trials x features x voxels]
features_full = pred_models[:,:,best_model_inds[rv,pp]]
# Take out the relevant features now
features_full = features_full[:,features_to_use,:]
# Note there may be some zeros in this matrix, if we used fewer than the max number of features.
# But they are zero in weight matrix too, so turns out ok.

_weights = torch_utils._to_torch(weights[rv,:,pp], device=device)   
_weights = _weights[:, features_to_use]
_bias = torch_utils._to_torch(bias[rv,pp], device=device)

print('number of zeros:')
print(np.sum(features_full[0,:,0]==0))

print('size of weights is:')
print(_weights.shape)

if features_mt is not None:
    _features_m = torch_utils._to_torch(features_mt[rv,:], device=device)
    _features_m = _features_m[:,features_to_use]
if features_st is not None:
    _features_s = torch_utils._to_torch(features_st[rv,:], device=device)
    _features_s = _features_s[:,features_to_use]

pred_block = np.full(fill_value=0, shape=(n_trials, lv), dtype=dtype)

# Now looping over validation set trials in batches
for rt, lt in numpy_utils.iterate_range(0, n_trials, sample_batch_size):

    _features = torch_utils._to_torch(features_full[rt,:], device=device) # trials x features
    if features_mt is not None:    
        # features_m is [nvoxels x nfeatures] - need [trials x features x voxels]
        _features = _features - torch.tile(torch.unsqueeze(_features_m, dim=0), [_features.shape[0], 1, 1]).moveaxis([1],[2])

    if features_st is not None:
        _features = _features/torch.tile(torch.unsqueeze(_features_s, dim=0), [_features.shape[0], 1, 1]).moveaxis([1],[2])
        _features[torch.isnan(_features)] = 0.0 # this applies in the pca case when last few columns of features are missing
        _features[torch.isinf(_features)] = 0.0
        
    # features is [#samples, #features, #voxels] - swap dims to [#voxels, #samples, features]
    _features = torch.transpose(torch.transpose(_features, 0, 2), 1, 2)
    # weights is [#voxels, #features]
    # _r will be [#voxels, #samples, 1] - then [#samples, #voxels]

    _r = torch.squeeze(torch.bmm(_features, torch.unsqueeze(_weights, 2)), dim=2).t() 

    if _bias is not None:
        _r = _r + torch.tile(torch.unsqueeze(_bias, 0), [_r.shape[0],1])

    pred_block[rt] = torch_utils.get_value(_r) 

# Now for this batch of voxels and this partial version of the model, measure performance.
#                 print('\nEvaluating correlation coefficient on validation set...\n')
for vi in range(lv):   
    val_cc[rv[vi],pp] = np.corrcoef(voxel_data[:,rv[vi]], pred_block[:,vi])[0,1]  
    val_r2[rv[vi],pp] = get_r2(voxel_data[:,rv[vi]], pred_block[:,vi])

sys.stdout.flush()


Evaluating version 3 of 7: texture_just_mean_magnitudes_no_other_modules
Includes 16 features
number of zeros:
0
size of weights is:
torch.Size([100, 16])


In [1008]:
_r

tensor([[-0.0001, -0.0002,  0.0127,  ..., -0.0003,  0.0088,  0.0046],
        [-0.0001, -0.0002,  0.0127,  ..., -0.0003,  0.0088,  0.0046],
        [-0.0001, -0.0002,  0.0127,  ..., -0.0003,  0.0088,  0.0046],
        ...,
        [-0.0001, -0.0002,  0.0127,  ..., -0.0003,  0.0088,  0.0046],
        [-0.0001, -0.0002,  0.0127,  ..., -0.0003,  0.0088,  0.0046],
        [-0.0001, -0.0002,  0.0127,  ..., -0.0003,  0.0088,  0.0046]],
       device='cuda:0')

In [1002]:
_bias.shape

torch.Size([100])

In [992]:
np.corrcoef(voxel_data[:,rv[vi]], pred_block[:,vi])


array([[ 1., nan],
       [nan, nan]])

In [996]:
pred_block[:,vi]

array([0.00455699, 0.00455699, 0.00455699, 0.00455699, 0.00455699,
       0.00455699, 0.00455699, 0.00455699, 0.00455699, 0.00455699,
       0.00455699, 0.00455699, 0.00455699, 0.00455699, 0.00455699,
       0.00455699, 0.00455699, 0.00455699, 0.00455699, 0.00455699,
       0.00455699, 0.00455699, 0.00455699, 0.00455699, 0.00455699,
       0.00455699, 0.00455699, 0.00455699, 0.00455699, 0.00455699,
       0.00455699, 0.00455699, 0.00455699, 0.00455699, 0.00455699,
       0.00455699, 0.00455699, 0.00455699, 0.00455699, 0.00455699,
       0.00455699, 0.00455699, 0.00455699, 0.00455699, 0.00455699,
       0.00455699, 0.00455699, 0.00455699, 0.00455699, 0.00455699,
       0.00455699, 0.00455699, 0.00455699, 0.00455699, 0.00455699,
       0.00455699, 0.00455699, 0.00455699, 0.00455699, 0.00455699,
       0.00455699, 0.00455699], dtype=float32)

In [995]:
voxel_data[:,rv[vi]]

array([-0.57709819, -0.68539327,  1.24508476,  1.55866957, -0.78992164,
        0.55576277,  0.2402944 , -0.44996917,  0.69701743, -0.45656109,
        0.49361086,  0.09621488,  0.13764949, -0.5610894 , -0.14203432,
        1.0124855 ,  1.45037448,  0.21863545,  1.28651929,  0.41921684,
        0.90701562, -0.24373759,  0.71867621,  0.40226632, -0.27198848,
       -0.22207853,  0.05383851,  0.48513556,  1.1433816 , -0.91516733,
       -0.21737012, -1.19579279,  0.63580698,  0.25347826,  0.23935276,
        1.8590709 , -0.28046378, -0.44243559,  0.8797065 , -0.28328881,
       -0.46786144, -0.47162822,  1.00118518,  0.3485896 ,  1.34302104,
       -0.06293189,  0.98705995, -0.20041959, -1.86439741,  0.33163908,
        0.36554012, -0.14674288, -0.08082404, -2.81551099, -1.78435326,
       -0.4979957 ,  2.39583826,  0.12823254,  1.42494869, -0.52812999,
       -1.26736188,  0.39849955])

In [967]:
torch.sum(_features_s==0, axis=1)

tensor([407, 407, 407, 407, 407, 402, 407, 407, 407, 407, 407, 402, 402, 402,
        402, 402, 402, 407, 407, 402, 407, 407, 402, 402, 407, 402, 402, 407,
        407, 402, 402, 402, 402, 407, 407, 407, 407, 407, 407, 407, 407, 407,
        407, 407, 407, 402, 407, 402, 407, 402, 402, 402, 407, 407, 407, 407,
        402, 402, 407, 402, 402, 402, 402, 407, 402, 407, 402, 402, 407, 407,
        407, 407, 402, 407, 407, 402, 407, 407, 407, 402, 407, 407, 407, 407,
        407, 407, 407, 407, 402, 402, 407, 402, 407, 407, 407, 407, 407, 407,
        402, 402], device='cuda:0')

In [948]:
np.sum(is_defined)

1041

In [949]:
is_defined

array([ True,  True,  True, ...,  True,  True,  True])

In [941]:
features_full[0,0:1000]

array([[-2.2392195e-01, -2.2392195e-01, -2.2392195e-01, ...,
        -2.2392195e-01, -3.9323947e-01, -3.9323947e-01],
       [ 1.4081092e-01,  1.4081092e-01,  1.4081092e-01, ...,
         1.4081092e-01,  7.1858428e-02,  7.1858428e-02],
       [ 2.5447050e-02,  2.5447050e-02,  2.5447050e-02, ...,
         2.5447050e-02, -8.5845679e-02, -8.5845679e-02],
       ...,
       [-4.5780726e-10, -4.5780726e-10, -4.5780726e-10, ...,
        -4.5780726e-10, -1.7213719e-10, -1.7213719e-10],
       [ 7.7769069e-10,  7.7769069e-10,  7.7769069e-10, ...,
         7.7769069e-10,  1.5812712e-11,  1.5812712e-11],
       [-9.0202068e-10, -9.0202068e-10, -9.0202068e-10, ...,
        -9.0202068e-10, -5.1483917e-10, -5.1483917e-10]], dtype=float32)

In [944]:
is_defined = feature_inds_defined_each_prf[masks[:,pp],pp]==1
is_defined.shape
~is_defined

array([False, False, False, ..., False, False, False])

In [937]:
_features = torch_utils._to_torch(features_full[rt,:], device=device)

In [938]:
f = _features[0,0,:]
w = _weights[0,:]

In [939]:
f[0:1000]

tensor([-0.2239, -0.2239, -0.2239, -0.2239, -0.2239, -0.3932, -0.2239, -0.2239,
        -0.2239, -0.2239, -0.2239, -0.3932, -0.3932, -0.3932, -0.3932, -0.3932,
        -0.3932, -0.2239, -0.2239, -0.3932, -0.2239, -0.2239, -0.3932, -0.3932,
        -0.2239, -0.3932, -0.3932, -0.2239, -0.2239, -0.3932, -0.3932, -0.3932,
        -0.3932, -0.2239, -0.2239, -0.2239, -0.2239, -0.2239, -0.2239, -0.2239,
        -0.2239, -0.2239, -0.2239, -0.2239, -0.2239, -0.3932, -0.2239, -0.3932,
        -0.2239, -0.3932, -0.3932, -0.3932, -0.2239, -0.2239, -0.2239, -0.2239,
        -0.3932, -0.3932, -0.2239, -0.3932, -0.3932, -0.3932, -0.3932, -0.2239,
        -0.3932, -0.2239, -0.3932, -0.3932, -0.2239, -0.2239, -0.2239, -0.2239,
        -0.3932, -0.2239, -0.2239, -0.3932, -0.2239, -0.2239, -0.2239, -0.3932,
        -0.2239, -0.2239, -0.2239, -0.2239, -0.2239, -0.2239, -0.2239, -0.2239,
        -0.3932, -0.3932, -0.2239, -0.3932, -0.2239, -0.2239, -0.2239, -0.2239,
        -0.2239, -0.2239, -0.3932, -0.39

In [903]:
w

tensor([ 5.9987e-04,  3.4266e-04, -6.4415e-05,  ...,  7.1773e-04,
         3.4391e-04,  5.9923e-04], device='cuda:0')

In [915]:
m = f*w
torch.any(torch.isnan(m))

tensor(False, device='cuda:0')

In [918]:
torch.any(torch.isinf(f))

tensor(True, device='cuda:0')

In [916]:
torch.sum(m)

tensor(nan, device='cuda:0')

In [893]:
_features.shape

torch.Size([100, 62, 1041])

In [895]:
torch.unsqueeze(_weights,2).shape

torch.Size([100, 1041, 1])

In [978]:
val_cc[0:10,:]

array([[-0.1396755 ,  0.0279291 , -0.14319251,  0.        , -0.14238772,
         0.0430938 , -0.13888815],
       [-0.03079309,  0.13824493, -0.05539858,  0.        , -0.0558647 ,
         0.15264574, -0.03141689],
       [ 0.33008984,  0.130216  ,  0.32965907,  0.        ,  0.33155468,
         0.14273554,  0.33203235],
       [ 0.15855937, -0.1713802 ,  0.16552094,  0.        ,  0.16560157,
        -0.17134535,  0.15864831],
       [ 0.0140794 ,  0.16797902,  0.00353686,  0.        ,  0.00367823,
         0.16828535,  0.01411854],
       [ 0.09249812,  0.00386856,  0.09289657,  0.        ,  0.09301239,
         0.00457587,  0.09261369],
       [ 0.15115827,  0.00928749,  0.15275912,  0.        ,  0.1525444 ,
         0.00997545,  0.15098521],
       [ 0.21069852, -0.06625794,  0.21573968,  0.        ,  0.21705034,
        -0.06592955,  0.21206613],
       [ 0.05650845,  0.08847162,  0.05080779,  0.        ,  0.05120433,
         0.06917796,  0.05689257],
       [ 0.09817851, -0.1558

In [971]:
val_cc, val_r2 = validate_fwrf_model(best_params, models, val_voxel_data, val_stim_data, _feature_extractor, \
                                   sample_batch_size=100, voxel_batch_size=100, debug=True, dtype=np.float32)


Clearing BDCN contour features from memory.
Clearing steerable pyramid features from memory.
Getting features for prf 0: [x,y,sigma] is [-0.55 -0.55 0.0400]
Running BDCN contour feature extraction...
Images array shape is:
(62, 1, 240, 240)
Final array shape is:
torch.Size([62, 1, 240, 240])
time elapsed = 0.64745
Getting features for pRF [x,y,sigma]:
[-0.55, -0.55, 0.03999999910593033]
bbox to crop is:
[232, 240, 0, 9]
[min max] of first image patch is:
[tensor(0.0058, device='cuda:0'), tensor(0.3294, device='cuda:0')]
Preparing for PCA: original dims of features:
(62, 72)
Applying pre-computed PCA matrix...
Running steerable pyramid feature extraction...
Images array shape is:
(62, 1, 240, 240)
time elapsed = 11.12420
Computing higher order correlations...
time elapsed = 0.76572
Final size of features concatenated is [62 x 641]
Feature types included are:
['pixel_stats', 'mean_magnitudes', 'mean_realparts', 'marginal_stats_lowpass_recons', 'variance_highpass_resid', 'magnitude_featur

In [970]:
def validate_fwrf_model(best_params, prf_models, voxel_data, images, _feature_extractor, \
                                   sample_batch_size=100, voxel_batch_size=100, debug=False, dtype=np.float32):
    
    """ 
    Evaluate trained model, leaving out a subset of features at a time.
    """
    
    params = best_params
    dtype = images.dtype.type
    device = _texture_fn.device
    
    n_trials, n_voxels = len(images), len(params[0])
    n_prfs = prf_models.shape[0]
    n_features = params[1].shape[1]  
    n_voxels = np.shape(voxel_data)[1]

    best_models, weights, bias, features_mt, features_st, best_model_inds = params
    masks, partial_version_names = _feature_extractor.get_partial_versions()
    masks = np.transpose(masks)    
    n_features_max = _feature_extractor.max_features
    n_partial_versions = len(partial_version_names)
    
    # val_cc is the correlation coefficient bw real and predicted responses across trials, for each voxel.
    val_cc  = np.zeros(shape=(n_voxels, n_partial_versions), dtype=dtype)
    val_r2 = np.zeros(shape=(n_voxels, n_partial_versions), dtype=dtype)

    pred_models = np.full(fill_value=0, shape=(n_trials, n_features_max, n_prfs), dtype=dtype)
    feature_inds_defined_each_prf = np.full(fill_value=0, shape=(n_features_max, n_prfs), dtype=bool)
    
    start_time = time.time()    
    with torch.no_grad():
        
        # First gather texture features for all pRFs.
        
        _feature_extractor.clear_maps()
        
        for mm in range(n_prfs):
            if mm>1 and debug:
                break
            print('Getting features for prf %d: [x,y,sigma] is [%.2f %.2f %.4f]'%(mm, prf_models[mm,0],  prf_models[mm,1],  prf_models[mm,2] ))
            all_feat_concat, feature_inds_defined = _feature_extractor(images, prf_models[mm,:], mm, fitting_mode=False)
            
            pred_models[:,feature_inds_defined,mm] = torch_utils.get_value(all_feat_concat)
            feature_inds_defined_each_prf[:,mm] = feature_inds_defined
            
        _feature_extractor.clear_maps()
        
        vv=-1
        ## Looping over voxels here in batches, will eventually go through all.
        for rv, lv in numpy_utils.iterate_range(0, n_voxels, voxel_batch_size):
            vv=vv+1
            print('Getting predictions for voxels [%d-%d] of %d'%(rv[0],rv[-1],n_voxels))

            if vv>1 and debug:
                break

            # Looping over versions of model w different features set to zero (variance partition)
            for pp in range(n_partial_versions):

        #         nonzero_inds_full = np.logical_and(masks[:,pp], feature_inds_defined[:,mm])             
        #         nonzero_inds_short = masks[feature_inds_defined,pp]==1

                print('\nEvaluating version %d of %d: %s'%(pp, n_partial_versions, partial_version_names[pp]))

                features_to_use = masks[:,pp]==1
                print('Includes %d features'%np.sum(features_to_use))

                # [trials x features x voxels]
                features_full = pred_models[:,:,best_model_inds[rv,pp]]
                # Take out the relevant features now
                features_full = features_full[:,features_to_use,:]
                # Note there may be some zeros in this matrix, if we used fewer than the max number of features.
                # But they are zero in weight matrix too, so turns out ok.

                _weights = torch_utils._to_torch(weights[rv,:,pp], device=device)   
                _weights = _weights[:, features_to_use]
                _bias = torch_utils._to_torch(bias[rv,pp], device=device)

                print('number of zeros:')
                print(np.sum(features_full[0,:,0]==0))

                print('size of weights is:')
                print(_weights.shape)

                if features_mt is not None:
                    _features_m = torch_utils._to_torch(features_mt[rv,:], device=device)
                    _features_m = _features_m[:,features_to_use]
                if features_st is not None:
                    _features_s = torch_utils._to_torch(features_st[rv,:], device=device)
                    _features_s = _features_s[:,features_to_use]

                pred_block = np.full(fill_value=0, shape=(n_trials, lv), dtype=dtype)

                # Now looping over validation set trials in batches
                for rt, lt in numpy_utils.iterate_range(0, n_trials, sample_batch_size):

                    _features = torch_utils._to_torch(features_full[rt,:], device=device) # trials x features
                    if features_mt is not None:    
                        # features_m is [nvoxels x nfeatures] - need [trials x features x voxels]
                        _features = _features - torch.tile(torch.unsqueeze(_features_m, dim=0), [_features.shape[0], 1, 1]).moveaxis([1],[2])

                    if features_st is not None:
                        _features = _features/torch.tile(torch.unsqueeze(_features_s, dim=0), [_features.shape[0], 1, 1]).moveaxis([1],[2])
                        # if any entries in std are zero or nan, this gives bad result - fix these now.
                        # these bad entries will also be zero in weights, so doesn't matter. just want to avoid nans.
                        _features[torch.isnan(_features)] = 0.0 
                        _features[torch.isinf(_features)] = 0.0
                        
                    # features is [#samples, #features, #voxels] - swap dims to [#voxels, #samples, features]
                    _features = torch.transpose(torch.transpose(_features, 0, 2), 1, 2)
                    # weights is [#voxels, #features]
                    # _r will be [#voxels, #samples, 1] - then [#samples, #voxels]

                    _r = torch.squeeze(torch.bmm(_features, torch.unsqueeze(_weights, 2)), dim=2).t() 

                    if _bias is not None:
                        _r = _r + torch.tile(torch.unsqueeze(_bias, 0), [_r.shape[0],1])

                    pred_block[rt] = torch_utils.get_value(_r) 

                # Now for this batch of voxels and this partial version of the model, measure performance.
        #                 print('\nEvaluating correlation coefficient on validation set...\n')
                for vi in range(lv):   
                    val_cc[rv[vi],pp] = np.corrcoef(voxel_data[:,rv[vi]], pred_block[:,vi])[0,1]  
                    val_r2[rv[vi],pp] = get_r2(voxel_data[:,rv[vi]], pred_block[:,vi])

                sys.stdout.flush()

    val_cc = np.nan_to_num(val_cc)
    val_r2 = np.nan_to_num(val_r2) 
    
    return val_cc, val_r2

In [ ]:
def validate_texture_model(best_params, prf_models, val_voxel_single_trial_data, val_stim_single_trial_data, _texture_fn, sample_batch_size=100, voxel_batch_size=100, debug=False, dtype=np.float32):
    
    # EVALUATE PERFORMANCE ON VALIDATION SET

    print('\nGetting model predictions on validation set...\n')
    val_voxel_pred = get_predictions_texture_model(val_stim_single_trial_data, _texture_fn, best_params, prf_models, sample_batch_size=sample_batch_size, voxel_batch_size=voxel_batch_size, debug=debug)

    # val_cc is the correlation coefficient bw real and predicted responses across trials, for each voxel.
    n_voxels = np.shape(val_voxel_single_trial_data)[1]
    val_cc  = np.zeros(shape=(n_voxels), dtype=dtype)
    val_r2 = np.zeros(shape=(n_voxels), dtype=dtype)
    
    print('\nEvaluating correlation coefficient on validation set...\n')
    for v in tqdm(range(n_voxels)):    
        val_cc[v] = np.corrcoef(val_voxel_single_trial_data[:,v], val_voxel_pred[:,v])[0,1]  
        val_r2[v] = get_r2(val_voxel_single_trial_data[:,v], val_voxel_pred[:,v])
        
    val_cc = np.nan_to_num(val_cc)
    val_r2 = np.nan_to_num(val_r2)    
    
    return val_cc, val_r2


def validate_texture_model_partial(best_params, prf_models, val_voxel_single_trial_data, val_stim_single_trial_data, _texture_fn, sample_batch_size=100, voxel_batch_size=100, debug=False, dtype=np.float32):
    
    """ 
    Evaluate trained model, leaving out a subset of features at a time.
    """
    
    # val_cc is the correlation coefficient bw real and predicted responses across trials, for each voxel.
    n_voxels = np.shape(val_voxel_single_trial_data)[1]
    n_feature_types = len(_texture_fn.feature_types_include)
    val_cc  = np.zeros(shape=(n_voxels, n_feature_types), dtype=dtype)
    val_r2 = np.zeros(shape=(n_voxels, n_feature_types), dtype=dtype)

    orig_feature_column_labels = _texture_fn.feature_column_labels
    orig_excluded_features = _texture_fn.feature_types_exclude

    for ff, feat_name in enumerate(_texture_fn.feature_types_include):

        print('\nVariance partition, leaving out: %s'%feat_name)
        _texture_fn.update_feature_list(orig_excluded_features+[feat_name])
        print('Remaining features are:')
        print(_texture_fn.feature_types_include)

        # Choose columns of interest here, leaving out weights for one feature at a time
        params_to_use = copy.deepcopy(best_params)
        columns_to_use = np.where(orig_feature_column_labels!=ff)[0]
        print(columns_to_use)
        params_to_use[1] = params_to_use[1][:,columns_to_use]
        if best_params[3] is not None:
            params_to_use[3] = params_to_use[3][:,columns_to_use]
            params_to_use[4] = params_to_use[4][:,columns_to_use]

        print(best_params[1].shape)
        print(params_to_use[1].shape)
        print('\nGetting model predictions on validation set...\n')
#         val_voxel_pred = get_predictions_texture_model(val_stim_single_trial_data, _texture_fn, best_params, prf_models, sample_batch_size=sample_batch_size, voxel_batch_size=voxel_batch_size, debug=debug)
        val_voxel_pred = get_predictions_texture_model(val_stim_single_trial_data, _texture_fn, params_to_use, prf_models, sample_batch_size=sample_batch_size, voxel_batch_size=voxel_batch_size, debug=debug)
        print('\nEvaluating correlation coefficient on validation set...\n')
        for v in range(n_voxels):    
            val_cc[v,ff] = np.corrcoef(val_voxel_single_trial_data[:,v], val_voxel_pred[:,v])[0,1]  
            val_r2[v,ff] = get_r2(val_voxel_single_trial_data[:,v], val_voxel_pred[:,v])

    val_cc = np.nan_to_num(val_cc)
    val_r2 = np.nan_to_num(val_r2) 
    
    return val_cc, val_r2



def get_predictions_texture_model(images, _texture_fn, params, prf_models, sample_batch_size=100, voxel_batch_size=100, debug=False):
   
    dtype = images.dtype.type
    device = _texture_fn.device

    best_models, weights, bias, features_mt, features_st, best_model_inds = params
        
    n_trials, n_voxels = len(images), len(params[0])
    n_prfs = prf_models.shape[0]
    n_features = params[1].shape[1]    
    
    pred = np.full(fill_value=0, shape=(n_trials, n_voxels), dtype=dtype)
    pred_models = np.full(fill_value=0, shape=(n_trials, n_features, n_prfs), dtype=dtype)
    
    start_time = time.time()    
    with torch.no_grad():
        
        # First gather texture features for all pRFs.
        for mm in range(n_prfs):
            if mm>1 and debug:
                break
            print('Getting features for prf %d: [x,y,sigma] is [%.2f %.2f %.4f]'%(mm, prf_models[mm,0],  prf_models[mm,1],  prf_models[mm,2] ))
            all_feat_concat, feature_info = _texture_fn(images,prf_models[mm,:])
            
            pred_models[:,:,mm] = torch_utils.get_value(all_feat_concat)
        
        vv=-1
        ## Looping over voxels here in batches, will eventually go through all.
        for rv, lv in numpy_utils.iterate_range(0, n_voxels, voxel_batch_size):
            vv=vv+1
            print('Getting predictions for voxels [%d-%d] of %d'%(rv[0],rv[-1],n_voxels))

            if vv>1 and debug:
                break
                
            # [trials x features x voxels]
            features = pred_models[:,:,best_model_inds[rv]]

            pred_block = np.full(fill_value=0, shape=(n_trials, lv), dtype=dtype)
            if features_mt is not None:
                _features_m = torch_utils._to_torch(features_mt[rv,:])
            if features_st is not None:
                _features_s = torch_utils._to_torch(features_st[rv,:])
            _weights = torch_utils._to_torch(weights[rv,:])
            _bias = torch_utils._to_torch(bias[rv])
                
            # Now looping over validation set trials in batches
            for rt, lt in numpy_utils.iterate_range(0, n_trials, sample_batch_size):

                _features = torch_utils._to_torch(features[rt,:,:]) # trials x features x voxels
                if features_mt is not None:    
                    # features_m is [nvoxels x nfeatures] - need [trials x features x voxels]
                    _features = _features - torch.tile(torch.unsqueeze(_features_m, dim=0), [_features.shape[0], 1, 1]).moveaxis([1],[2])

                if features_st is not None:
                    _features = _features/torch.tile(torch.unsqueeze(_features_s, dim=0), [_features.shape[0], 1, 1]).moveaxis([1],[2])
                    _features[torch.isnan(_features)] = 0.0 # this applies in the pca case when last few columns of features are missing

                # features is [#samples, #features, #voxels] - swap dims to [#voxels, #samples, features]
                _features = torch.transpose(torch.transpose(_features, 0, 2), 1, 2)
                # weights is [#voxels, #features]
                # _r will be [#voxels, #samples, 1] - then [#samples, #voxels]

                _r = torch.squeeze(torch.bmm(_features, torch.unsqueeze(_weights, 2)), dim=2).t() 

                if _bias is not None:
                    _r = _r + torch.tile(torch.unsqueeze(_bias, 0), [_r.shape[0],1])

                pred_block[rt] = torch_utils.get_value(_r) 
                
            pred[:,rv] = pred_block
            sys.stdout.flush()
            
    total_time = time.time() - start_time
    print ('\n---------------------------------------')
    print ('total time = %fs' % total_time)
    print ('sample throughput = %fs/sample' % (total_time / n_trials))
    print ('voxel throughput = %fs/voxel' % (total_time / n_voxels))
    sys.stdout.flush()
    return pred

   
def validate_bdcn_model(best_params, prf_models, val_voxel_single_trial_data, \
                            val_stim_single_trial_data, _feature_extractor, pc=None, sample_batch_size=100, \
                            voxel_batch_size=100, debug=False, dtype=np.float32):
    
    """ 
    Evaluate trained model, leaving out a subset of features at a time.
    """
    print('starting validation function')
    sys.stdout.flush()
        
    images = val_stim_single_trial_data
    params = best_params
    dtype = images.dtype.type
    device = _feature_extractor.device
    
    n_trials, n_voxels = len(images), len(params[0])
    n_prfs = prf_models.shape[0]
    n_features = params[1].shape[1]  

    best_models, weights, bias, features_mt, features_st, best_model_inds = params
    if pc is not None:
        pca_wts, pct_var_expl, min_pct_var, n_comp_needed, pca_pre_mean = pc
    
    n_voxels = np.shape(val_voxel_single_trial_data)[1]
    print('starting to initialize big arrays')
    sys.stdout.flush()
        
    val_cc  = np.zeros(shape=(n_voxels, 1), dtype=dtype)
    val_r2 = np.zeros(shape=(n_voxels, 1), dtype=dtype)
    n_features_actual = np.zeros(shape=(n_prfs,), dtype=int)

    pred_models = np.full(fill_value=0, shape=(n_trials, n_features, n_prfs), dtype=dtype)
    
    print('about to start loop')
    sys.stdout.flush()
        
    start_time = time.time()    
    with torch.no_grad():
        
        # First gather features for all pRFs.
        
        _feature_extractor.clear_maps()
        
        sys.stdout.flush()
        
        
        for mm in range(n_prfs):
            if mm>1 and debug:
                break
            print('Getting features for prf %d: [x,y,sigma] is [%.2f %.2f %.4f]'%(mm, prf_models[mm,0],  prf_models[mm,1],  prf_models[mm,2] ))
            
            features = _feature_extractor(images, prf_models[mm,:]).detach().cpu().numpy()   

            if pc is not None:
                print('Applying pre-computed PCA matrix')
                # Apply the PCA transformation, just as it was done during training
                nfeat = features.shape[1]
                features_submean = features - np.tile(np.expand_dims(pca_pre_mean[mm][0:nfeat], axis=0), [n_trials, 1])
                features_reduced = features_submean @ np.transpose(pca_wts[mm][0:n_comp_needed[mm],0:nfeat])                                               
                features = features_reduced

            n_features_actual[mm] = features.shape[1]

            pred_models[:,0:n_features_actual[mm],mm] = features
            
            sys.stdout.flush()
        
                
        _feature_extractor.clear_maps()
        
        sys.stdout.flush()
        
    
        vv=-1
        ## Looping over voxels here in batches, will eventually go through all.
        for rv, lv in numpy_utils.iterate_range(0, n_voxels, voxel_batch_size):
            vv=vv+1
            print('Getting predictions for voxels [%d-%d] of %d'%(rv[0],rv[-1],n_voxels))

            if vv>1 and debug:
                break
            
            # [trials x features x voxels]
            # to keep this from being huge, just keep the maximum number of features needed for any voxel
            # there will be some zeros still, but they are also zero in the weights so not a problem.
            feat2use = np.max(n_features_actual[best_model_inds[rv]])
            features = pred_models[:,0:feat2use,best_model_inds[rv]]

            print('size of feature matrix to use is:')
            print(features.shape)
            
            _weights = torch_utils._to_torch(weights[rv,0:feat2use]) 
            _bias = torch_utils._to_torch(bias[rv])

            if features_mt is not None:
                _features_m = torch_utils._to_torch(features_mt[rv,0:feat2use])
            if features_st is not None:
                _features_s = torch_utils._to_torch(features_st[rv,0:feat2use])

            pred_block = np.full(fill_value=0, shape=(n_trials, lv), dtype=dtype)

            # Now looping over validation set trials in batches
            for rt, lt in numpy_utils.iterate_range(0, n_trials, sample_batch_size):

                _features = torch_utils._to_torch(features[rt,:,:]) # trials x features x voxels
                if features_mt is not None:    
                    # features_m is [nvoxels x nfeatures] - need [trials x features x voxels]
                    _features = _features - torch.tile(torch.unsqueeze(_features_m, dim=0), [_features.shape[0], 1, 1]).moveaxis([1],[2])

                if features_st is not None:
                    _features = _features/torch.tile(torch.unsqueeze(_features_s, dim=0), [_features.shape[0], 1, 1]).moveaxis([1],[2])
                    _features[torch.isnan(_features)] = 0.0 # this applies in the pca case when last few columns of features are missing

                # features is [#samples, #features, #voxels] - swap dims to [#voxels, #samples, features]
                _features = torch.transpose(torch.transpose(_features, 0, 2), 1, 2)
                # weights is [#voxels, #features]
                # _r will be [#voxels, #samples, 1] - then [#samples, #voxels]

                _r = torch.squeeze(torch.bmm(_features, torch.unsqueeze(_weights, 2)), dim=2).t() 

                if _bias is not None:
                    _r = _r + torch.tile(torch.unsqueeze(_bias, 0), [_r.shape[0],1])

                pred_block[rt] = torch_utils.get_value(_r) 

            # Now for this batch of voxels and this partial version of the model, measure performance.
#                 print('\nEvaluating correlation coefficient on validation set...\n')
            for vi in range(lv):   
                val_cc[rv[vi],0] = np.corrcoef(val_voxel_single_trial_data[:,rv[vi]], pred_block[:,vi])[0,1]  
                val_r2[rv[vi],0] = get_r2(val_voxel_single_trial_data[:,rv[vi]], pred_block[:,vi])

            sys.stdout.flush()
        
    val_cc = np.nan_to_num(val_cc)
    val_r2 = np.nan_to_num(val_r2) 
    
    return val_cc, val_r2

    